In [ ]:
# C:\Users\SBA\github\langchain-kr\TechReader_gayoon\techreader_data\LLM_TechLibrary.pdf
# C:\Users\SBA\github\langchain-kr\TechReader_gayoon\techreader_data\RAG비법노트.pdf

In [1]:
# 📌 RAG 파이프라인 (TechLibrary.pdf)
# 실행 전: pip install langchain langchain-community langchain-openai gradio faiss-cpu

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import gradio as gr

# =========================
# 1. 환경 변수 로드
# =========================
load_dotenv()  # .env 안에 OPENAI_API_KEY 있어야 함

# =========================
# 2. PDF 로드
# =========================
pdf_path = r"C:\Users\SBA\github\langchain-kr\TechReader_gayoon\techreader_data\LLM_TechLibrary.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()

# =========================
# 3. 텍스트 분할
# =========================
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50, separators=["\n", " ", ""]
)
splits = text_splitter.split_documents(docs)

# =========================
# 4. 임베딩 & 벡터DB 생성
# =========================
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore = FAISS.from_documents(splits, embeddings)

# =========================
# 5. Retriever + LLM 구성
# =========================
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

template = """
당신은 질문-답변 AI 어시스턴트입니다.
아래 문맥(context)을 바탕으로 질문(question)에 답하세요.
만약 문맥에 답이 없으면 "모르겠다"고 말하세요.

#Context:
{context}

#Question:
{question}

#Answer:
"""

prompt = PromptTemplate(
    input_variables=["context", "question"], template=template
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

# =========================
# 6. Gradio UI
# =========================
def chat(query):
    response = qa_chain.run(query)
    return response

iface = gr.Interface(
    fn=chat,
    inputs=gr.Textbox(lines=2, placeholder="질문을 입력하세요..."),
    outputs="text",
    title="📘 TechLibrary RAG Chatbot",
    description="20쪽짜리 TechLibrary 보고서를 기반으로 질의응답합니다."
)

if __name__ == "__main__":
    iface.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


C:\Users\SBA\AppData\Local\Temp\ipykernel_6540\2308884681.py:76: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


Created dataset file at: .gradio\flagged\dataset1.csv


In [2]:
# 📌 RAG 파이프라인 (출처 페이지 표시)
# 실행 전: pip install langchain langchain-community langchain-openai gradio faiss-cpu

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import gradio as gr

# =========================
# 1. 환경 변수 로드
# =========================
load_dotenv()

# =========================
# 2. PDF 로드
# =========================
pdf_path = r"C:\Users\SBA\github\langchain-kr\TechReader_gayoon\techreader_data\LLM_TechLibrary.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()

# =========================
# 3. 텍스트 분할
# =========================
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50, separators=["\n", " ", ""]
)
splits = text_splitter.split_documents(docs)

# =========================
# 4. 임베딩 & 벡터DB 생성
# =========================
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore = FAISS.from_documents(splits, embeddings)

# =========================
# 5. Retriever + LLM 구성
# =========================
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

template = """
당신은 질문-답변 AI 어시스턴트입니다.
아래 문맥(context)을 바탕으로 질문(question)에 답하세요.
만약 문맥에 답이 없으면 "모르겠다"고 말하세요.

#Context:
{context}

#Question:
{question}

#Answer:
"""

prompt = PromptTemplate(
    input_variables=["context", "question"], template=template
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True   # ✅ 출처 문서 반환
)

# =========================
# 6. Gradio UI (답변 + 출처 페이지 표시)
# =========================
def chat(query):
    result = qa_chain(query)   # dict 반환
    answer = result["result"]

    # 참고한 페이지 번호 모으기
    pages = sorted(set([doc.metadata.get("page", "N/A") + 1 for doc in result["source_documents"]]))
    sources = ", ".join([f"{p}쪽" for p in pages])

    return f"📖 답변:\n{answer}\n\n📌 참고 페이지: {sources}"

iface = gr.Interface(
    fn=chat,
    inputs=gr.Textbox(lines=2, placeholder="질문을 입력하세요..."),
    outputs="text",
    title="📘 TechLibrary RAG Chatbot",
    description="20쪽짜리 TechLibrary 보고서를 기반으로 질의응답 + 출처 페이지 표시"
)

if __name__ == "__main__":
    iface.launch()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


C:\Users\SBA\AppData\Local\Temp\ipykernel_6540\2771694091.py:76: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain(query)   # dict 반환


In [3]:
# 📌 TechLibrary RAG + 예상 질문 리스트 UI
import os
from dotenv import load_dotenv
import gradio as gr
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# =========================
# 1. 환경 변수 로드
# =========================
load_dotenv()

# =========================
# 2. PDF 로드
# =========================
pdf_path = r"C:\Users\SBA\github\langchain-kr\TechReader_gayoon\techreader_data\LLM_TechLibrary.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()

# =========================
# 3. 텍스트 분할
# =========================
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

# =========================
# 4. 벡터 DB
# =========================
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore = FAISS.from_documents(splits, embeddings)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# =========================
# 5. QA Chain
# =========================
template = """
당신은 질문-답변 AI 어시스턴트입니다.
아래 문맥(context)을 바탕으로 질문(question)에 답하세요.
답이 문맥에 없으면 "모르겠다"고 하세요.

#Context:
{context}

#Question:
{question}

#Answer:
"""
prompt = PromptTemplate(input_variables=["context", "question"], template=template)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

# =========================
# 6. 예상 질문 자동 생성 (보고서 기반)
# =========================
def generate_example_questions(n=5):
    text_sample = "\n".join([doc.page_content for doc in splits[:3]])  # 처음 몇 페이지 샘플
    prompt = f"""
    다음 보고서를 기반으로 사용자가 물어볼 만한 예상 질문 {n}개를 만들어 주세요.
    각 질문은 간결하게 작성하세요.
    보고서 내용: {text_sample}
    """
    return llm.invoke(prompt).content.split("\n")

example_questions = generate_example_questions(5)

# =========================
# 7. 질문 처리 함수
# =========================
def answer_question(query):
    result = qa_chain(query)
    answer = result["result"]
    pages = sorted(set([doc.metadata.get("page", 0) + 1 for doc in result["source_documents"]]))
    sources = ", ".join([f"{p}쪽" for p in pages])
    return f"💡 질문: {query}\n\n📖 답변:\n{answer}\n\n📌 참고 페이지: {sources}"

# =========================
# 8. Gradio UI
# =========================
with gr.Blocks() as demo:
    gr.Markdown("## 📘 TechLibrary RAG Chatbot\n예상 질문을 클릭하거나 직접 질문을 입력하세요.")

    with gr.Row():
        with gr.Column():
            question_buttons = [gr.Button(q) for q in example_questions]
        with gr.Column():
            query_box = gr.Textbox(placeholder="직접 질문을 입력해 보세요...")
            submit_btn = gr.Button("질문하기")
    
    output = gr.Textbox(label="답변", lines=10)

    # 버튼 클릭 이벤트
    for btn in question_buttons:
        btn.click(fn=answer_question, inputs=gr.Textbox(value=btn.value, visible=False), outputs=output)

    # 직접 질문 입력 이벤트
    submit_btn.click(fn=answer_question, inputs=query_box, outputs=output)

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


가윤님이 원하시는 건 문서를 업로드 → 자동으로 청킹 & 벡터DB 구축 → 예상 질문 생성 → 바로 Q&A 가능
즉, 완전한 문서 업로드 기반 RAG 서비스네요.

Gradio에서 gr.File을 사용하면 사용자가 PDF를 올리면,

PyPDFLoader → split → embedding → vectorstore 생성

문서 내용 기반 예상 질문 생성

버튼 & 입력창에서 바로 Q&A

이렇게 파이프라인이 연결됩니다.

In [5]:
import os
from dotenv import load_dotenv
import gradio as gr
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

load_dotenv()

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# ===== RAG 체인 생성 함수 =====
def build_rag_pipeline(pdf_path, n_questions=5):
    # PDF 로드
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()

    # 청킹
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    splits = text_splitter.split_documents(docs)

    # 벡터DB
    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    vectorstore = FAISS.from_documents(splits, embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    # QA 체인
    template = """
    당신은 질문-답변 AI 어시스턴트입니다.
    아래 문맥(context)을 바탕으로 질문(question)에 답하세요.
    답이 문맥에 없으면 "모르겠다"고 하세요.

    #Context:
    {context}

    #Question:
    {question}

    #Answer:
    """
    prompt = PromptTemplate(input_variables=["context", "question"], template=template)
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )

    # 예상 질문 자동 생성
    sample_text = "\n".join([doc.page_content for doc in splits[:3]])
    question_prompt = f"""
    다음 문서를 기반으로 사용자가 물어볼 만한 예상 질문 {n_questions}개를 만들어 주세요.
    각 질문은 간결하게 작성하세요.
    문서 내용:
    {sample_text}
    """
    example_questions = llm.invoke(question_prompt).content.split("\n")
    example_questions = [q.strip("- ").strip() for q in example_questions if q.strip()]

    return qa_chain, example_questions


# ===== 질문 처리 함수 =====
def answer_question(query, qa_chain):
    result = qa_chain(query)
    answer = result["result"]
    pages = sorted(set([doc.metadata.get("page", 0) + 1 for doc in result["source_documents"]]))
    sources = ", ".join([f"{p}쪽" for p in pages])
    return f"💡 질문: {query}\n\n📖 답변:\n{answer}\n\n📌 참고 페이지: {sources}"


# ===== Gradio UI =====
with gr.Blocks() as demo:
    gr.Markdown("## 📘 TechReader RAG\n문서를 업로드하면 자동으로 Q&A 시스템이 준비됩니다.")

    uploaded_file = gr.File(label="📂 PDF 업로드", file_types=[".pdf"])
    status = gr.Markdown("⏳ 문서를 업로드하면 파이프라인이 생성됩니다.")

    question_buttons = gr.Column()   # 동적으로 버튼 생성할 공간
    query_box = gr.Textbox(placeholder="직접 질문을 입력하세요...")
    submit_btn = gr.Button("질문하기")
    output = gr.Textbox(label="답변", lines=10)

    # 내부 상태
    qa_chain_state = gr.State()

    # 문서 업로드 이벤트
    def process_file(file):
        qa_chain, example_questions = build_rag_pipeline(file.name)
        btns = [gr.Button(q) for q in example_questions[:5]]
        return qa_chain, "✅ 파이프라인 생성 완료! 예상 질문이 준비되었습니다.", btns

    uploaded_file.upload(process_file, inputs=uploaded_file, outputs=[qa_chain_state, status, question_buttons])

    # 직접 질문
    submit_btn.click(answer_question, inputs=[query_box, qa_chain_state], outputs=output)

    # 버튼 클릭 → 답변
    # 버튼은 동적 생성이므로 upload 단계에서 이벤트 바인딩 필요

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\SBA\github\langchain-kr\.venv\Lib\site-packages\gradio\queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\SBA\github\langchain-kr\.venv\Lib\site-packages\gradio\route_utils.py", line 349, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\SBA\github\langchain-kr\.venv\Lib\site-packages\gradio\blocks.py", line 2284, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\SBA\github\langchain-kr\.venv\Lib\site-packages\gradio\blocks.py", line 2057, in postprocess_data
    raise InvalidComponentError(
gradio.exceptions.InvalidComponentError: <class 'gradio.layouts.column.Column'> Component not a valid output component

# 1차 성공 
원하는 문서를 업로드하면 RAG 파이프라인이 빠르게 작동하여 예상 질문까시 생성해주며, 답변 출처(몇 페이지)까지 제공해주는 기능 

In [6]:
import os
from dotenv import load_dotenv
import gradio as gr
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

load_dotenv()

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# ===== RAG 체인 생성 함수 =====
def build_rag_pipeline(pdf_path, n_questions=5):
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    splits = text_splitter.split_documents(docs)

    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    vectorstore = FAISS.from_documents(splits, embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    template = """
    당신은 질문-답변 AI 어시스턴트입니다.
    아래 문맥(context)을 바탕으로 질문(question)에 답하세요.
    답이 문맥에 없으면 "모르겠다"고 하세요.

    #Context:
    {context}

    #Question:
    {question}

    #Answer:
    """
    prompt = PromptTemplate(input_variables=["context", "question"], template=template)
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )

    # 예상 질문 생성
    sample_text = "\n".join([doc.page_content for doc in splits[:3]])
    question_prompt = f"""
    다음 문서를 기반으로 사용자가 물어볼 만한 예상 질문 {n_questions}개를 만들어 주세요.
    각 질문은 간결하게 작성하세요.
    문서 내용:
    {sample_text}
    """
    example_questions = llm.invoke(question_prompt).content.split("\n")
    example_questions = [q.strip("- ").strip() for q in example_questions if q.strip()]

    return qa_chain, example_questions


# ===== 질문 처리 함수 =====
def answer_question(query, qa_chain):
    result = qa_chain(query)
    answer = result["result"]
    pages = sorted(set([doc.metadata.get("page", 0) + 1 for doc in result["source_documents"]]))
    sources = ", ".join([f"{p}쪽" for p in pages])
    return f"💡 질문: {query}\n\n📖 답변:\n{answer}\n\n📌 참고 페이지: {sources}"


# ===== Gradio UI =====
with gr.Blocks() as demo:
    gr.Markdown("## 📘 TechReader RAG\n문서를 업로드하면 자동으로 Q&A 시스템이 준비됩니다.")

    uploaded_file = gr.File(label="📂 PDF 업로드", file_types=[".pdf"])
    status = gr.Markdown("⏳ 문서를 업로드하면 파이프라인이 생성됩니다.")

    question_selector = gr.Radio(label="예상 질문 선택", choices=[])
    query_box = gr.Textbox(placeholder="직접 질문을 입력하세요...")
    submit_btn = gr.Button("질문하기")
    output = gr.Textbox(label="답변", lines=10)

    qa_chain_state = gr.State()

    # 문서 업로드 이벤트
    def process_file(file):
        qa_chain, example_questions = build_rag_pipeline(file.name)
        return qa_chain, "✅ 파이프라인 생성 완료!", gr.update(choices=example_questions)

    uploaded_file.upload(process_file, inputs=uploaded_file, outputs=[qa_chain_state, status, question_selector])

    # 직접 질문
    submit_btn.click(answer_question, inputs=[query_box, qa_chain_state], outputs=output)

    # 예상 질문 선택
    question_selector.change(answer_question, inputs=[question_selector, qa_chain_state], outputs=output)

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


# 2차 시도 - 
예상 질문을 다각적으로 생성하기 위해 멀티벡터스토어 리트리버 사용  

한 문서를 여러 벡터로 표현해, 더 정밀하고 다양한 방식으로 검색할 수 있는 고급 리트리버다. 
(청크, 요약, 가설 쿼리) 

단, 예상 질문 생성시 페이지별 질문 생성, 중복 질문 발생 시, 캐시에 저장해뒀다가 재질문 시, 불러오는 식으로 구현 


=> 
PDF를 페이지 단위로 잘라서 LLM에게 이 페이지 내용 기반으로 질문 2-3개 만들어달라고 요처 ㅇ
각 페이지별 질문을 모아 리스트에 저장
SET이나 해시를 활용해 중복 질문 제거 

=> 질문 캐싱 구현
예상 질문을 KEY, 생성 답변을 VALUE로 저장하는 딕셔너리 캐시 활용 
사용자가 질문할 시, 캐시에 해당 질문이 있으면 저장된 답변 즉시 반환
없으면 답변 생성 후 캐시에 저장




In [7]:
import os
from dotenv import load_dotenv
import gradio as gr
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.storage import InMemoryStore
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

load_dotenv()

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# ===== RAG 파이프라인 (MultiVectorRetriever 기반) =====
def build_rag_pipeline(pdf_path, n_questions=5):
    # 1️⃣ 문서 로드
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()

    # 2️⃣ 청크 분할
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = splitter.split_documents(docs)

    # 3️⃣ 벡터스토어 + InMemoryStore
    vectorstore = FAISS.from_documents(chunks, embeddings)
    store = InMemoryStore()

    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key="doc_id"
    )

    # 부모 문서 저장
    for i, doc in enumerate(docs):
        retriever.docstore.mset([(str(i), doc)])

    # 4️⃣ 요약 임베딩 추가
    for i, chunk in enumerate(chunks[:10]):  # 처음 10개 청크만 예시
        summary = llm.invoke(f"다음 내용을 한 문장으로 요약:\n{chunk.page_content}").content
        retriever.vectorstore.add_texts([summary], metadatas=[{"doc_id": str(i)}])

    # 5️⃣ 예상 질문 임베딩 추가
    sample_text = "\n".join([doc.page_content for doc in chunks[:3]])
    q_prompt = f"다음 문서를 기반으로 사용자가 물어볼만한 질문 {n_questions}개를 생성해 주세요:\n{sample_text}"
    example_questions = llm.invoke(q_prompt).content.split("\n")
    example_questions = [q.strip("- ").strip() for q in example_questions if q.strip()]

    for i, q in enumerate(example_questions):
        retriever.vectorstore.add_texts([q], metadatas=[{"doc_id": "0"}])  # doc_id=0 (대표)

    # 6️⃣ RetrievalQA 체인 구성
    template = """
    당신은 질문-답변 AI 어시스턴트입니다.
    아래 문맥(context)을 바탕으로 질문(question)에 답하세요.
    답이 문맥에 없으면 "모르겠다"고 하세요.

    #Context:
    {context}

    #Question:
    {question}

    #Answer:
    """
    prompt = PromptTemplate(input_variables=["context", "question"], template=template)
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )

    return qa_chain, example_questions


# ===== 질문 처리 함수 =====
def answer_question(query, qa_chain):
    result = qa_chain(query)
    answer = result["result"]
    pages = sorted(set([doc.metadata.get("page", 0) + 1 for doc in result["source_documents"]]))
    sources = ", ".join([f"{p}쪽" for p in pages])
    return f"💡 질문: {query}\n\n📖 답변:\n{answer}\n\n📌 참고 페이지: {sources}"


# ===== Gradio UI =====
with gr.Blocks() as demo:
    gr.Markdown("## 📘 TechReader RAG (MultiVectorRetriever)\n문서를 업로드하면 자동으로 Q&A 시스템이 준비됩니다.")

    uploaded_file = gr.File(label="📂 PDF 업로드", file_types=[".pdf"])
    status = gr.Markdown("⏳ 문서를 업로드하면 파이프라인이 생성됩니다.")

    question_selector = gr.Radio(label="예상 질문 선택", choices=[])
    query_box = gr.Textbox(placeholder="직접 질문을 입력하세요...")
    submit_btn = gr.Button("질문하기")
    output = gr.Textbox(label="답변", lines=10)

    qa_chain_state = gr.State()

    # 문서 업로드 이벤트
    def process_file(file):
        qa_chain, example_questions = build_rag_pipeline(file.name)
        return qa_chain, "✅ 파이프라인 생성 완료!", gr.update(choices=example_questions)

    uploaded_file.upload(process_file, inputs=uploaded_file, outputs=[qa_chain_state, status, question_selector])

    # 직접 질문
    submit_btn.click(answer_question, inputs=[query_box, qa_chain_state], outputs=output)

    # 예상 질문 선택
    question_selector.change(answer_question, inputs=[question_selector, qa_chain_state], outputs=output)

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


# 3차 성공 - 페이지별 예상 질문 리스트 생성 완료 

In [8]:
import os
from dotenv import load_dotenv
import gradio as gr
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

load_dotenv()

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# ===== 전역 캐시 =====
cache = {}

# ===== RAG 체인 생성 함수 =====
def build_rag_pipeline(pdf_path, n_questions_per_page=2):
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    splits = text_splitter.split_documents(docs)

    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    vectorstore = FAISS.from_documents(splits, embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    template = """
    당신은 질문-답변 AI 어시스턴트입니다.
    아래 문맥(context)을 바탕으로 질문(question)에 답하세요.
    답이 문맥에 없으면 "모르겠다"고 하세요.

    #Context:
    {context}

    #Question:
    {question}

    #Answer:
    """
    prompt = PromptTemplate(input_variables=["context", "question"], template=template)
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )

    # ===== 페이지별 예상 질문 생성 =====
    example_questions = []
    for doc in docs:
        page_content = doc.page_content[:1500]  # 길이 제한 (토큰 절약)
        question_prompt = f"""
        다음 페이지 내용을 기반으로 사용자가 물어볼 만한 질문 {n_questions_per_page}개를 만들어 주세요.
        각 질문은 반드시 이 페이지 내용과 직접적으로 관련되도록 해주세요.
        페이지 내용:
        {page_content}
        """
        q_text = llm.invoke(question_prompt).content.split("\n")
        q_list = [q.strip("- ").strip() for q in q_text if q.strip()]
        example_questions.extend(q_list)

    # 중복 제거
    example_questions = list(dict.fromkeys(example_questions))  

    return qa_chain, example_questions


# ===== 질문 처리 함수 =====
def answer_question(query, qa_chain):
    # 캐시 확인
    if query in cache:
        return cache[query]

    result = qa_chain(query)
    answer = result["result"]
    pages = sorted(set([doc.metadata.get("page", 0) + 1 for doc in result["source_documents"]]))
    sources = ", ".join([f"{p}쪽" for p in pages])
    response = f"💡 질문: {query}\n\n📖 답변:\n{answer}\n\n📌 참고 페이지: {sources}"

    # 캐시에 저장
    cache[query] = response
    return response


# ===== Gradio UI =====
with gr.Blocks() as demo:
    gr.Markdown("## 📘 TechReader RAG\n문서를 업로드하면 자동으로 Q&A 시스템이 준비됩니다. (페이지별 질문 생성 + 캐싱)")

    uploaded_file = gr.File(label="📂 PDF 업로드", file_types=[".pdf"])
    status = gr.Markdown("⏳ 문서를 업로드하면 파이프라인이 생성됩니다.")

    question_selector = gr.Radio(label="예상 질문 선택", choices=[])
    query_box = gr.Textbox(placeholder="직접 질문을 입력하세요...")
    submit_btn = gr.Button("질문하기")
    output = gr.Textbox(label="답변", lines=10)

    qa_chain_state = gr.State()

    def process_file(file):
        qa_chain, example_questions = build_rag_pipeline(file.name)
        return qa_chain, "✅ 파이프라인 생성 완료!", gr.update(choices=example_questions)

    uploaded_file.upload(process_file, inputs=uploaded_file, outputs=[qa_chain_state, status, question_selector])

    # 직접 질문
    submit_btn.click(answer_question, inputs=[query_box, qa_chain_state], outputs=output)

    # 예상 질문 선택
    question_selector.change(answer_question, inputs=[question_selector, qa_chain_state], outputs=output)

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


<img src="img/3.png" alt="샘플 이미지">


# 4차 시도 - 예상 질문의 가독성 높이기 
영구 저장 DB로 쓰고 싶어, 지금 예상 질문 리스트를 많이 생성해준 것은 좋아, 하지만 사용자 화면에 이렇게 많이 나열해두면 가독성이 떨어지잖아, PDF 문서를 올리면 해당 PDF의 목차 정보에 따른 핵심 주제를 파악해서, 핵심 주제를 선택했을 때 예상 질문이 5-6개 나오는 걸로 바꾸고 싶어 

+ 추가 개선 설계 아이디어
+ 사용자에게 핵심 주제 리스트 보여주고, 선택하면 해당 섹션의 예상 질문 5-6개 출력 
+ 목차 기반 > 섹션 선택 > 질문 출력 > 답변 

In [ ]:
1. SQLite 스레드 에러 
Gradio는 비동기/멀티스레드로 동작하므로 sqlite3.connect()시 check_same_thread=False 값을 줘야한다.  

2. pdf 목차 기반 섹션별 질문 생성 개선 
fitz(pymupdf)를 사용하면 pdf outline 목차를 추출할 수 있다. 
outline이 없으면 llm으로 섹션 요약을 생성해 가짜 목차를 만드는 fallback을 넣을 수 있다. 
이후 섹션별로 청킹 > 벡터 db 저장 > 해당 섹션 기준으로 예상 질문 생성 한다. 

In [23]:
import os
import sqlite3 
import fitz  # PyMuPDF
import gradio as gr
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

load_dotenv()

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# ===== SQLite 캐시 (스레드 안전) =====
conn = sqlite3.connect("qa_cache.db", check_same_thread=False)
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS questions_cache (
    pdf_name TEXT,
    section TEXT,
    question TEXT,
    answer TEXT,
    PRIMARY KEY(pdf_name, section, question)
)
""")
conn.commit()

def get_cached_answer(pdf_name, section, question):
    cur.execute("SELECT answer FROM questions_cache WHERE pdf_name=? AND section=? AND question=?",
                (pdf_name, section, question))
    row = cur.fetchone()
    return row[0] if row else None

def save_answer(pdf_name, section, question, answer):
    cur.execute("INSERT OR REPLACE INTO questions_cache VALUES (?, ?, ?, ?)",
                (pdf_name, section, question, answer))
    conn.commit()

# ===== 목차 추출 함수 =====
def extract_pdf_sections(pdf_path):
    doc = fitz.open(pdf_path)
    toc = doc.get_toc(simple=True)  # (level, title, page)
    sections = []

    if toc:
        for level, title, page in toc:
            if level == 1:
                sections.append((title.strip(), page))
    else:
        text = ""
        for page in doc.pages(0, min(5, len(doc))):
            text += page.get_text()
        prompt = f"""
        다음 문서를 4~5개의 핵심 주제로 나눠 주세요.
        주제만 짧게 나열하세요.
        문서 내용:
        {text}
        """
        response = llm.invoke(prompt).content.split("\n")
        sections = [(r.strip("- ").strip(), 0) for r in response if r.strip()]
    doc.close()
    return sections

# ===== RAG 파이프라인 (섹션 단위) =====
def build_section_rag(pdf_path, section, start_page, end_page=None):
    doc = fitz.open(pdf_path)
    text = ""
    end_page = end_page or len(doc)
    for i in range(start_page-1, end_page):
        text += doc[i].get_text()
    doc.close()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    splits = text_splitter.create_documents([text])

    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    vectorstore = FAISS.from_documents(splits, embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    template = """
    당신은 질문-답변 AI 어시스턴트입니다.
    아래 문맥(context)을 바탕으로 질문(question)에 답하세요.
    답이 문맥에 없으면 "모르겠다"고 하세요.

    #Context:
    {context}

    #Question:
    {question}

    #Answer:
    """
    prompt = PromptTemplate(input_variables=["context", "question"], template=template)

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )
    return qa_chain

# ===== 예상 질문 생성 =====
def generate_section_questions(pdf_path, section, start_page, n_questions=5):
    doc = fitz.open(pdf_path)
    text = ""
    for i in range(start_page-1, min(start_page+2, len(doc))):
        text += doc[i].get_text()
    doc.close()

    prompt = f"""
    아래 문서를 기반으로 '{section}' 주제와 관련된 예상 질문 {n_questions}개를 작성하세요.
    ⚠️ 조건:
    - 반드시 문서 내용에서 추출 가능한 질문만 포함할 것
    - "물론입니다", "네" 같은 불필요한 접두 문구는 절대 쓰지 말 것
    - 질문은 한 줄씩, 번호와 마침표 없이 깔끔하게 출력할 것

    문서 내용:
    {text}
    """
    raw_output = llm.invoke(prompt).content
    qs = raw_output.split("\n")

    # 후처리: 불필요 문구 제거 + "물론입니다", "네" 등 걸러내기
    clean_qs = []
    for q in qs:
        q = q.strip("- ").strip()
        if q and not any(bad in q for bad in ["물론입니다", "네 "]):
            clean_qs.append(q)

    return clean_qs


# ===== 질문 처리 =====
def answer_question(pdf_path, section, query, qa_chain):
    if not qa_chain:
        return "⚠️ QA 체인이 준비되지 않았습니다. 먼저 섹션을 선택하세요."

    pdf_name = os.path.basename(pdf_path)
    cached = get_cached_answer(pdf_name, section, query)
    if cached:
        return cached

    try:
        result = qa_chain(query)
    except Exception as e:
        return f"⚠️ 답변 생성 중 오류 발생: {e}"

    answer = result["result"]
    pages = sorted(set([doc.metadata.get("page", 0) + 1 for doc in result["source_documents"]]))
    sources = ", ".join([f"{p}쪽" for p in pages]) if pages else "출처 없음"
    response = f"💡 질문: {query}\n\n📖 답변:\n{answer}\n\n📌 참고 페이지: {sources}"

    save_answer(pdf_name, section, query, response)
    return response


# ===== Gradio UI =====
with gr.Blocks() as demo:
    gr.Markdown("## 📘 TechReader RAG\nPDF를 업로드하면 목차 기반 섹션 선택 → 예상 질문 → 답변 생성")

    uploaded_file = gr.File(label="📂 PDF 업로드", file_types=[".pdf"])
    status = gr.Markdown("⏳ 문서 업로드 대기 중...")
    section_selector = gr.Dropdown(label="핵심 주제(섹션) 선택", choices=[])
    question_selector = gr.Radio(label="예상 질문 선택", choices=[])
    query_box = gr.Textbox(placeholder="직접 질문을 입력하세요...")
    submit_btn = gr.Button("질문하기")
    output = gr.Textbox(label="답변", lines=10)

    pdf_state = gr.State()
    section_state = gr.State()
    qa_chain_state = gr.State()

    # 파일 업로드 → 섹션 추출
    def process_file(file):
        sections = extract_pdf_sections(file.name)
        return file.name, "✅ 문서 분석 완료!", gr.update(choices=[s[0] for s in sections])

    uploaded_file.upload(process_file, inputs=uploaded_file, outputs=[pdf_state, status, section_selector])

    # 섹션 선택 → 예상 질문 생성
    def process_section(pdf_path, section):
        sections = extract_pdf_sections(pdf_path)
        matches = [(t, p) for (t, p) in sections if t.strip().lower() == section.strip().lower()]
        if not matches:
            return gr.update(choices=["⚠️ 섹션을 찾을 수 없습니다."]), None, section

        _, start_page = matches[0]
        qa_chain = build_section_rag(pdf_path, section, start_page)
        qs = generate_section_questions(pdf_path, section, start_page)
        return gr.update(choices=qs), qa_chain, section

    section_selector.change(process_section, inputs=[pdf_state, section_selector], outputs=[question_selector, qa_chain_state, section_state])

    # 직접 질문
    submit_btn.click(answer_question, inputs=[pdf_state, section_state, query_box, qa_chain_state], outputs=output)

    # 예상 질문 선택
    question_selector.change(answer_question, inputs=[pdf_state, section_state, question_selector, qa_chain_state], outputs=output)

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.


In [ ]:
# 이렇게 하면 선택한 섹션 범위만 청킹돼서 vectorstore에 들어가니까, 출처 페이지도 해당 섹션 범위 내에서만 나오게 됩니다.


# 5차 

In [27]:
import os
import sqlite3 
import fitz  # PyMuPDF
import gradio as gr
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

load_dotenv()

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# ===== SQLite 캐시 (스레드 안전) =====
conn = sqlite3.connect("qa_cache.db", check_same_thread=False)
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS questions_cache (
    pdf_name TEXT,
    section TEXT,
    question TEXT,
    answer TEXT,
    PRIMARY KEY(pdf_name, section, question)
)
""")
conn.commit()

def get_cached_answer(pdf_name, section, question):
    cur.execute("SELECT answer FROM questions_cache WHERE pdf_name=? AND section=? AND question=?",
                (pdf_name, section, question))
    row = cur.fetchone()
    return row[0] if row else None

def save_answer(pdf_name, section, question, answer):
    cur.execute("INSERT OR REPLACE INTO questions_cache VALUES (?, ?, ?, ?)",
                (pdf_name, section, question, answer))
    conn.commit()

# ===== 목차 추출 함수 =====
def extract_pdf_sections(pdf_path):
    doc = fitz.open(pdf_path)
    toc = doc.get_toc(simple=True)  # (level, title, page)
    sections = []

    if toc:
        level1_sections = [(title.strip(), page) for level, title, page in toc if level == 1]
        for i, (title, start_page) in enumerate(level1_sections):
            end_page = level1_sections[i+1][1] - 1 if i+1 < len(level1_sections) else len(doc)
            sections.append((title, start_page, end_page))
    else:
        text = "".join(doc[i].get_text() for i in range(min(5, len(doc))))
        prompt = f"""
        다음 문서를 4~5개의 핵심 주제로 나눠 주세요.
        주제만 짧게 나열하세요.
        문서 내용:
        {text}
        """
        response = llm.invoke(prompt).content.split("\n")
        sections = [(r.strip("- ").strip(), 1, len(doc)) for r in response if r.strip()]
    doc.close()
    return sections

# ===== RAG 파이프라인 (섹션 단위) =====
# ===== RAG 파이프라인 (섹션 단위) =====
def build_section_rag(pdf_path, section, start_page, end_page):
    doc = fitz.open(pdf_path)
    texts, docs = [], []
    for i in range(start_page-1, end_page):
        page_text = doc[i].get_text()
        if page_text.strip():
            docs.append({"page": i+1, "text": page_text})
    doc.close()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    splits = []
    for d in docs:
        chunks = text_splitter.create_documents([d["text"]], metadatas=[{"page": d["page"]}])
        splits.extend(chunks)

    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    vectorstore = FAISS.from_documents(splits, embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    template = """
    당신은 질문-답변 AI 어시스턴트입니다.
    아래 문맥(context)을 바탕으로 질문(question)에 답하세요.
    답이 문맥에 없으면 "모르겠다"고 하세요.

    #Context:
    {context}

    #Question:
    {question}

    #Answer:
    """
    prompt = PromptTemplate(input_variables=["context", "question"], template=template)

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )
    return qa_chain

# ===== 예상 질문 생성 =====
def generate_section_questions(pdf_path, section, start_page, n_questions=5):
    doc = fitz.open(pdf_path)
    text = ""
    for i in range(start_page-1, min(start_page+2, len(doc))):
        text += doc[i].get_text()
    doc.close()

    prompt = f"""
    아래 문서를 기반으로 '{section}' 주제와 관련된 예상 질문 {n_questions}개를 작성하세요.
    ⚠️ 조건:
    - 반드시 문서 내용에서 추출 가능한 질문만 포함할 것
    - "물론입니다", "네" 같은 불필요한 접두 문구는 절대 쓰지 말 것
    - 질문은 한 줄씩, 번호와 마침표 없이 깔끔하게 출력할 것

    문서 내용:
    {text}
    """
    raw_output = llm.invoke(prompt).content
    qs = raw_output.split("\n")

    # 후처리: 불필요 문구 제거 + "물론입니다", "네" 등 걸러내기
    clean_qs = []
    for q in qs:
        q = q.strip("- ").strip()
        if q and not any(bad in q for bad in ["물론입니다", "네 "]):
            clean_qs.append(q)

    return clean_qs


# ===== 질문 처리 =====
def answer_question(pdf_path, section, query, qa_chain):
    if not qa_chain:
        return "⚠️ QA 체인이 준비되지 않았습니다. 먼저 섹션을 선택하세요."

    pdf_name = os.path.basename(pdf_path)
    cached = get_cached_answer(pdf_name, section, query)
    if cached:
        return cached

    result = qa_chain(query)
    answer = result["result"]

    # ✅ source_documents에 page metadata 반영
    pages = sorted(set([doc.metadata.get("page") for doc in result["source_documents"] if "page" in doc.metadata]))
    sources = ", ".join([f"{p}쪽" for p in pages]) if pages else "출처 없음"

    response = f"💡 질문: {query}\n\n📖 답변:\n{answer}\n\n📌 참고 페이지: {sources}"

    save_answer(pdf_name, section, query, response)
    return response


# ===== Gradio UI =====
with gr.Blocks() as demo:
    gr.Markdown("## 📘 TechReader RAG\nPDF를 업로드하면 목차 기반 섹션 선택 → 예상 질문 → 답변 생성")

    uploaded_file = gr.File(label="📂 PDF 업로드", file_types=[".pdf"])
    status = gr.Markdown("⏳ 문서 업로드 대기 중...")
    section_selector = gr.Dropdown(label="핵심 주제(섹션) 선택", choices=[])
    question_selector = gr.Radio(label="예상 질문 선택", choices=[])
    query_box = gr.Textbox(placeholder="직접 질문을 입력하세요...")
    submit_btn = gr.Button("질문하기")
    output = gr.Textbox(label="답변", lines=10)

    pdf_state = gr.State()
    section_state = gr.State()
    qa_chain_state = gr.State()

    # 파일 업로드 → 섹션 추출
    def process_file(file):
        sections = extract_pdf_sections(file.name)
        return file.name, "✅ 문서 분석 완료!", gr.update(choices=[s[0] for s in sections])

    uploaded_file.upload(process_file, inputs=uploaded_file, outputs=[pdf_state, status, section_selector])

    # 섹션 선택 → 예상 질문 생성
    def process_section(pdf_path, section):
        sections = extract_pdf_sections(pdf_path)
        #  # ✅ (제목, 시작, 끝) 구조로 언패킹
        matches = [(t, s, e) for (t, s, e) in sections if t.strip().lower() == section.strip().lower()]
        if not matches:
            return gr.update(choices=["⚠️ 섹션을 찾을 수 없습니다."]), None, section

        _, start_page, end_page = matches[0]
        qa_chain = build_section_rag(pdf_path, section, start_page, end_page)
        qs = generate_section_questions(pdf_path, section, start_page)
        return gr.update(choices=qs), qa_chain, section
    
    section_selector.change(process_section, inputs=[pdf_state, section_selector], outputs=[question_selector, qa_chain_state, section_state])

    # 직접 질문
    submit_btn.click(answer_question, inputs=[pdf_state, section_state, query_box, qa_chain_state], outputs=output)

    # 예상 질문 선택
    question_selector.change(answer_question, inputs=[pdf_state, section_state, question_selector, qa_chain_state], outputs=output)

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.


✅ 코드의 장점

* 범용성: 목차 없는 PDF도 대응 (LLM으로 주제 생성).
* 효율성: SQLite 캐시로 반복 질문 최적화.
* 정확성: metadata.page 기반으로 실제 답변 출처 페이지 표시.
* 사용성: 예상 질문 리스트 → 사용자가 빠르게 시작 가능.
* UI 구조화: Gradio로 깔끔하게 PDF→섹션→질문→답변 흐름. 

⚠️ 개선 아이디어

* 다중 벡터 표현 (MultiVectorRetriever)
청크 + 요약 + 예상질문 임베딩을 동시에 넣어 검색 정확도를 높일 수 있음.

* 출처 강조
답변 내 출처 문장을 하이라이팅해 주면 신뢰성이 더 강화됨.

* DB 확장
SQLite 대신 Postgres 같은 영구 DB 사용하면 다중 사용자 환경에서도 안정적.

* 성능 최적화
20페이지 이하 문서는 지금 방식 충분.
수백 페이지 이상일 경우 → RAPTOR, ContextualCompressionRetriever 같은 고급 전략 고려.

# 6차 
캐시 유지과 리트리버 확장 = 멀티벡터스토어 리트리버 

In [ ]:
결론: "굳이 바꿀 필요가 있을까?"
대부분의 일반적인 캐시 시나리오(하나의 애플리케이션 내에서 빠른 데이터 조회를 위함)에서는 SQLite를 PostgreSQL로 바꾸는 것은 장점보다 단점이 훨씬 큽니다. 
속도 저하, 시스템 복잡성 증가, 관리 비용 발생 등의 문제가 따르기 때문입니다.

따라서 현재 SQLite 캐시 시스템이 성능이나 기능적으로 명확한 한계에 부딪힌 것이 아니라면, 
그대로 유지하는 것이 현명한 선택입니다. 

만약 여러 서버 간 캐시 공유나 높은 동시 쓰기 처리가 꼭 필요한 상황이라면, 
PostgreSQL보다는 Redis나 Memcached 같은 인메모리(In-memory) 캐시 전문 솔루션을 먼저 검토하는 것이 일반적이고 훨씬 효율적입니다.

In [ ]:
# 리트리버 방식	해결하는 문제	추천 단계
1. 단일 리트리버 + 메타데이터	(현재 문제) 섹션 선택의 불편함, 정보 단절	1순위 (강력 추천)
2. Self-Querying 리트리버	복잡하고 구조적인 질문 처리, LLM 기반 동적 검색	2순위 (고급 기능)
3. 멀티 벡터 리트리버	검색 정확도 및 답변 품질의 심층 최적화	3순위 (성능 고도화)

In [37]:
import os
import sqlite3
import fitz  # PyMuPDF
import gradio as gr
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain.schema import Document

# ==========================
# 기본 설정
# ==========================
load_dotenv()
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# ==========================
# SQLite 캐시 (질문-답변 저장)
# ==========================
conn = sqlite3.connect("qa_cache.db", check_same_thread=False)
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS questions_cache (
    pdf_name TEXT,
    section TEXT,
    question TEXT,
    answer TEXT,
    PRIMARY KEY(pdf_name, section, question)
)
""")
conn.commit()

def get_cached_answer(pdf_name, section, question):
    cur.execute("SELECT answer FROM questions_cache WHERE pdf_name=? AND section=? AND question=?",
                (pdf_name, section, question))
    row = cur.fetchone()
    return row[0] if row else None

def save_answer(pdf_name, section, question, answer):
    cur.execute("INSERT OR REPLACE INTO questions_cache VALUES (?, ?, ?, ?)",
                (pdf_name, section, question, answer))
    conn.commit()

# ==========================
# 목차 추출 함수
# ==========================
def extract_pdf_sections(pdf_path):
    doc = fitz.open(pdf_path)
    toc = doc.get_toc(simple=True)  # (level, title, page)
    sections = []

    if toc:
        level1_sections = [(title.strip(), page) for level, title, page in toc if level == 1]
        for i, (title, start_page) in enumerate(level1_sections):
            end_page = level1_sections[i+1][1] - 1 if i+1 < len(level1_sections) else len(doc)
            sections.append((title, start_page, end_page))
    else:
        text = "".join(doc[i].get_text() for i in range(min(5, len(doc))))
        prompt = f"""
        다음 문서를 4~5개의 핵심 주제로 나눠 주세요.
        주제만 짧게 나열하세요.
        문서 내용:
        {text}
        """
        response = llm.invoke(prompt).content.split("\n")
        sections = [(r.strip("- ").strip(), 1, len(doc)) for r in response if r.strip()]
    doc.close()
    return sections

# ==========================
# MultiVectorRetriever 구성
# ==========================
def build_multi_vector_retriever(pdf_path, section, start_page, end_page):
    doc = fitz.open(pdf_path)
    docs = []
    for i in range(start_page-1, end_page):
        page_text = doc[i].get_text()
        if page_text.strip():
            docs.append(Document(page_content=page_text, metadata={"page": i+1, "section": section}))
    doc.close()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    splits = text_splitter.split_documents(docs)

    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    vectorstore = FAISS.from_documents(splits, embeddings)

    # ✅ byte_store 추가 (필수)
    byte_store = InMemoryByteStore()

    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        byte_store=byte_store,
        id_key="section"
    )

    return retriever

# ==========================
# 예상 질문 생성
# ==========================
def generate_section_questions(pdf_path, section, start_page, n_questions=5):
    doc = fitz.open(pdf_path)
    text = ""
    for i in range(start_page-1, min(start_page+2, len(doc))):
        text += doc[i].get_text()
    doc.close()

    prompt = f"""
    아래 문서를 기반으로 '{section}' 주제와 관련된 예상 질문 {n_questions}개를 작성하세요.
    ⚠️ 조건:
    - 반드시 문서 내용에서 추출 가능한 질문만 포함할 것
    - "물론입니다", "네" 같은 불필요한 접두 문구는 절대 쓰지 말 것
    - 질문은 한 줄씩, 번호와 마침표 없이 깔끔하게 출력할 것

    문서 내용:
    {text}
    """
    raw_output = llm.invoke(prompt).content
    qs = raw_output.split("\n")

    clean_qs = []
    for q in qs:
        q = q.strip("- ").strip()
        if q and not any(bad in q for bad in ["물론입니다", "네 "]):
            clean_qs.append(q)

    return clean_qs

# ==========================
# 질문 처리
# ==========================
from langchain.chains import LLMChain
from langchain.schema import Document

def answer_question(pdf_path, section, query, retriever):
    if not retriever:
        return "⚠️ 리트리버가 준비되지 않았습니다. 먼저 섹션을 선택하세요."

    pdf_name = os.path.basename(pdf_path)
    cached = get_cached_answer(pdf_name, section, query)
    if cached:
        return cached

    # 🔹 검색
    docs = retriever.get_relevant_documents(query)
    context = "\n\n".join([d.page_content for d in docs])

    # 🔹 프롬프트
    template = """
    당신은 질문-답변 AI 어시스턴트입니다.
    아래 문맥(context)을 바탕으로 질문(question)에 답하세요.
    답이 문맥에 없으면 "모르겠다"고 하세요.

    #Context:
    {context}

    #Question:
    {question}

    #Answer:
    """
    prompt = PromptTemplate(input_variables=["context", "question"], template=template)
    chain = LLMChain(llm=llm, prompt=prompt)

    # 🔹 실행
    answer = chain.invoke({"context": context, "question": query})["text"]

    # 🔹 출처 페이지
    pages = sorted(set([doc.metadata.get("page") for doc in docs if "page" in doc.metadata]))
    sources = ", ".join([f"{p}쪽" for p in pages]) if pages else "출처 없음"

    response = f"💡 질문: {query}\n\n📖 답변:\n{answer}\n\n📌 참고 페이지: {sources}"
    save_answer(pdf_name, section, query, response)
    return response


# ==========================
# Gradio UI
# ==========================
with gr.Blocks() as demo:
    gr.Markdown("## 📘 TechReader RAG (MultiVectorRetriever + SQLite Cache)")

    uploaded_file = gr.File(label="📂 PDF 업로드", file_types=[".pdf"])
    status = gr.Markdown("⏳ 문서 업로드 대기 중...")
    section_selector = gr.Dropdown(label="핵심 주제(섹션) 선택", choices=[])
    question_selector = gr.Radio(label="예상 질문 선택", choices=[])
    query_box = gr.Textbox(placeholder="직접 질문을 입력하세요...")
    submit_btn = gr.Button("질문하기")
    output = gr.Textbox(label="답변", lines=10)

    pdf_state = gr.State()
    section_state = gr.State()
    retriever_state = gr.State()

    def process_file(file):
        sections = extract_pdf_sections(file.name)
        return file.name, "✅ 문서 분석 완료!", gr.update(choices=[s[0] for s in sections])

    uploaded_file.upload(process_file, inputs=uploaded_file, outputs=[pdf_state, status, section_selector])

    def process_section(pdf_path, section):
        sections = extract_pdf_sections(pdf_path)
        matches = [(t, s, e) for (t, s, e) in sections if t.strip().lower() == section.strip().lower()]
        if not matches:
            return gr.update(choices=["⚠️ 섹션을 찾을 수 없습니다."]), None, section

        _, start_page, end_page = matches[0]
        retriever = build_multi_vector_retriever(pdf_path, section, start_page, end_page)
        qs = generate_section_questions(pdf_path, section, start_page)
        return gr.update(choices=qs), retriever, section

    section_selector.change(process_section, inputs=[pdf_state, section_selector],
                            outputs=[question_selector, retriever_state, section_state])

    submit_btn.click(answer_question, inputs=[pdf_state, section_state, query_box, retriever_state], outputs=output)
    question_selector.change(answer_question, inputs=[pdf_state, section_state, question_selector, retriever_state], outputs=output)

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7888
* To create a public link, set `share=True` in `launch()`.


# 7차 : 검색 성능과 품질을 올리기 위해 멀티벡터리트리버 사용 

요약 임베딩과 가설 쿼리까지 통합한 멀티벡터스토어리트리버 

* 원본 : Faiss는 페이스북에서 000년도에 000한 개발자와 함께 개발한 라이브러리로~ 000한 약자의 의미를 지니고 있습니다. 
* 요약 : Faiss는 벡터스토어 라이브러리다. 
* 가설쿼리 : Faiss란 무엇인가? 어떤 기능을 제공하나요? 

=>
원본 청크 + 요약본 + 가설 질문 -> 여러 벡터를 연결해서 저장
원문 chunk 임베딩

chunk 요약본 임베딩

예상 질문(가설 쿼리) 임베딩 



In [ ]:
import os
import sqlite3
import fitz  # PyMuPDF
import gradio as gr
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

load_dotenv()

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# ===== SQLite 캐시 =====
conn = sqlite3.connect("qa_cache.db", check_same_thread=False)
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS questions_cache (
    pdf_name TEXT,
    section TEXT,
    question TEXT,
    answer TEXT,
    PRIMARY KEY(pdf_name, section, question)
)
""")
conn.commit()

def get_cached_answer(pdf_name, section, question):
    cur.execute(
        "SELECT answer FROM questions_cache WHERE pdf_name=? AND section=? AND question=?",
        (pdf_name, section, question),
    )
    row = cur.fetchone()
    return row[0] if row else None

def save_answer(pdf_name, section, question, answer):
    cur.execute(
        "INSERT OR REPLACE INTO questions_cache VALUES (?, ?, ?, ?)",
        (pdf_name, section, question, answer),
    )
    conn.commit()

# ===== 목차 추출 =====
def extract_pdf_sections(pdf_path):
    doc = fitz.open(pdf_path)
    toc = doc.get_toc(simple=True)
    sections = []

    if toc:
        level1_sections = [(title.strip(), page) for level, title, page in toc if level == 1]
        for i, (title, start_page) in enumerate(level1_sections):
            end_page = level1_sections[i+1][1] - 1 if i+1 < len(level1_sections) else len(doc)
            sections.append((title, start_page, end_page))
    else:
        # fallback: LLM으로 주제 생성
        text = "".join(doc[i].get_text() for i in range(min(5, len(doc))))
        prompt = f"다음 문서를 4~5개의 핵심 주제로 나눠 주세요:\n{text}"
        response = llm.invoke(prompt).content.split("\n")
        sections = [(r.strip("- ").strip(), 1, len(doc)) for r in response if r.strip()]
    doc.close()
    return sections

# ===== MultiVectorRetriever 생성 =====
def build_multi_vector_retriever(pdf_path, section, start_page, end_page, n_hypo=5):
    doc = fitz.open(pdf_path)
    section_texts, docs = [], []
    for i in range(start_page-1, end_page):
        page_text = doc[i].get_text()
        if page_text.strip():
            docs.append({"page": i+1, "text": page_text})
    doc.close()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    splits = []
    for d in docs:
        chunks = text_splitter.create_documents([d["text"]], metadatas=[{"page": d["page"]}])
        splits.extend(chunks)

    # 원문 청크 임베딩
    vectorstore = FAISS.from_documents(splits, embeddings)

    # 섹션 요약문
    summary_prompt = f"이 문서 섹션의 핵심 요약을 3문장으로 작성:\n{docs[0]['text'][:1000]}"
    summary = llm.invoke(summary_prompt).content
    summary_doc = text_splitter.create_documents([summary], metadatas=[{"page": start_page}])
    vectorstore.add_documents(summary_doc)

    # 가설 질문 생성
    hypo_prompt = f"""
    '{section}' 섹션을 기반으로 사용자가 물어볼 만한 질문 {n_hypo}개 작성.
    반드시 문서 내용에서 답변 가능한 질문만.
    질문은 간단하게, 한 줄씩.
    """
    raw_qs = llm.invoke(hypo_prompt).content.split("\n")
    hypo_questions = [q.strip("- ").strip() for q in raw_qs if q.strip()]

    hypo_docs = text_splitter.create_documents(
        hypo_questions, metadatas=[{"page": start_page, "section": section}]
    )
    vectorstore.add_documents(hypo_docs)

    # MultiVectorRetriever 구성
    retriever = MultiVectorRetriever(vectorstore=vectorstore, id_key="section")

    return retriever, hypo_questions

# ===== QA 체인 생성 =====
def build_qa_chain(retriever):
    template = """
    당신은 질문-답변 AI 어시스턴트입니다.
    아래 문맥(context)을 바탕으로 질문(question)에 답하세요.
    답이 문맥에 없으면 "모르겠다"고 하세요.

    #Context:
    {context}

    #Question:
    {question}

    #Answer:
    """
    prompt = PromptTemplate(input_variables=["context", "question"], template=template)

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True,
    )
    return qa_chain

# ===== 질문 처리 =====
def answer_question(pdf_path, section, query, qa_chain):
    if not qa_chain:
        return "⚠️ QA 체인이 준비되지 않았습니다. 먼저 섹션을 선택하세요."

    pdf_name = os.path.basename(pdf_path)
    cached = get_cached_answer(pdf_name, section, query)
    if cached:
        return cached

    result = qa_chain(query)
    answer = result["result"]

    pages = sorted(set([doc.metadata.get("page") for doc in result["source_documents"] if "page" in doc.metadata]))
    sources = ", ".join([f"{p}쪽" for p in pages]) if pages else "출처 없음"

    response = f"💡 질문: {query}\n\n📖 답변:\n{answer}\n\n📌 참고 페이지: {sources}"
    save_answer(pdf_name, section, query, response)
    return response

# ===== Gradio UI =====
with gr.Blocks() as demo:
    gr.Markdown("## 📘 TechReader RAG (MultiVectorRetriever)\nPDF → 섹션 선택 → 예상 질문 → 답변")

    uploaded_file = gr.File(label="📂 PDF 업로드", file_types=[".pdf"])
    status = gr.Markdown("⏳ 문서 업로드 대기 중...")
    section_selector = gr.Dropdown(label="핵심 주제(섹션) 선택", choices=[])
    question_selector = gr.Radio(label="예상 질문 선택", choices=[])
    query_box = gr.Textbox(placeholder="직접 질문을 입력하세요...")
    submit_btn = gr.Button("질문하기")
    output = gr.Textbox(label="답변", lines=10)

    pdf_state = gr.State()
    section_state = gr.State()
    qa_chain_state = gr.State()

    def process_file(file):
        sections = extract_pdf_sections(file.name)
        return file.name, "✅ 문서 분석 완료!", gr.update(choices=[s[0] for s in sections])

    uploaded_file.upload(process_file, inputs=uploaded_file, outputs=[pdf_state, status, section_selector])

    def process_section(pdf_path, section):
        sections = extract_pdf_sections(pdf_path)
        matches = [(t, s, e) for (t, s, e) in sections if t.strip().lower() == section.strip().lower()]
        if not matches:
            return gr.update(choices=["⚠️ 섹션을 찾을 수 없습니다."]), None, section

        _, start_page, end_page = matches[0]
        retriever, hypo_questions = build_multi_vector_retriever(pdf_path, section, start_page, end_page)
        qa_chain = build_qa_chain(retriever)
        return gr.update(choices=hypo_questions), qa_chain, section

    section_selector.change(process_section, inputs=[pdf_state, section_selector], outputs=[question_selector, qa_chain_state, section_state])

    submit_btn.click(answer_question, inputs=[pdf_state, section_state, query_box, qa_chain_state], outputs=output)
    question_selector.change(answer_question, inputs=[pdf_state, section_state, question_selector, qa_chain_state], outputs=output)

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7890
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\SBA\github\langchain-kr\.venv\Lib\site-packages\gradio\queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\SBA\github\langchain-kr\.venv\Lib\site-packages\gradio\route_utils.py", line 349, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\SBA\github\langchain-kr\.venv\Lib\site-packages\gradio\blocks.py", line 2274, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\SBA\github\langchain-kr\.venv\Lib\site-packages\gradio\blocks.py", line 1781, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\SBA\github\langchain-kr\.venv\Lib\site-packages\anyio\to_thread.py", line 56, in run_

# 8차 0903 1차 피드백 

In [ ]:
한 페이지의 document 객체 생성 > llm 에게 해당 페이지의 제목 추출해줘 
제목을 추출하는 것도 하나의 태스크

pdf 문서가 잘 올라오는지 요소별로 확인해라 
글자별 로딩 확인

llm에게 해당 document를 어떻게 제목을 뽑을 것인지 > 프롬프트를 통해 원샷 
제목을 만들어주는 파이프라인 중간에 하나 

split 가기 전에 제목 추출 
title과 subtitle을 만들라 
subtile을 기반으로 목차 페이지를 알려줘라 

=> 해당 문서의 패턴을 집중하는 rag를 만들라. 

In [ ]:
# 1. 제목과 소제목 추출하기 - 하나의 태스크 
라마 PARSER를 이용해 MD 파일로 변환
제목과 소제목은 # 으로 표시되어 일반 텍스트와 차이 

In [2]:
import nest_asyncio
nest_asyncio.apply()

from llama_parse import LlamaParse

parser = LlamaParse(result_type="markdown", language="ko")

file_path = r"techreader_data\LLM_TechLibrary.pdf"
parsed_docs = parser.load_data(file_path=file_path)  # 이제 정상 실행됨

docs = [doc.to_langchain_format() for doc in parsed_docs]
print(docs[0].page_content[:500])  # 일부 미리보기


Started parsing the file under job_id 962bce92-f498-4e68-9aca-d92a8d2d6d49

# LLM 이후를 설계하다

# 생성형 AI의 과제와 대안 찾기

# Tech Trend

“아는 것만 아는” LLM, 오히려 혁신을 저해한다

LLM을 학습한 추출 모델, 작아도 위험은 동일

# Tech Guide

LLM 한계 극복을 위한 RAG의 역할과 최신 동향

잊어버려야 할 것은 잊는 LLM이 필요한 시점

AI 코딩, LLM 혼합 전략이 답이다

무단 전재 본 PDF 문서는 IDG Korea의 자산으로, 저작권법의 보호를 받습니다.

재배포 금지 IDG Korea의 허락 없이 PDF 문서를 온라인 사이트 등에 무단 게재, 전재하거나 유포할 수 없습니다.





In [8]:
import os
from llama_parse import LlamaParse

# PDF 파서 초기화
parser = LlamaParse(
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="gemini-2.5-pro",
    vendor_multimodal_api_key=os.environ["GOOGLE_API_KEY"],
    result_type="markdown",
    parsing_mode="Unstructured",
    language="ko",
    parsing_instruction="""
     당신은 PDF 문서를 구조화된 Markdown으로 변환하는 파서입니다.
     
    가장 중요한 규칙:
    모든 텍스트는 가능한 모두 출력해주세요. 
    첫 페이지를 제외한 Tech Guide와 Tech Trend는 소제목이 아닙니다. 
    
    
    변환 규칙:
    0. 원문 텍스트는 가능한 한 모두 보존하세요. 
    1. 문서의 '주요 제목'은 반드시 `# 제목` 형식으로 추출하세요.
       - 제목 바로 아래 줄에 '저자 | 소속'이 있으면 `Author: 이름 | 소속`으로 출력하세요.
    2. 본문 내의 소제목은 `## 소제목`으로 변환하세요. 
       - 단, '# 1.' 같은 번호 형식, 첫 페이지를 제외한 영어 한 단어, 결론을 제외한 짧은 음절(예: 비추천 용도, 최적 용도, 주의점)은 소제목으로 간주하지 마세요.
    3. 제목/소제목 외의 일반 문단은 그냥 텍스트로 출력하세요. 특정 페이지에 일반 문단만 있어도 그대로 출력하세요. # 표시하지 마세요. 
    4. 일반 문단은 그냥 텍스트로 출력하되 • 표시로 시작하는 것도 그대로 출력해주세요.   
    5. 모든 출력은 순수한 Markdown 형식으로 작성하세요. 불필요한 설명, 번역, 해설은 절대 추가하지 마세요. 텍스트를 요약하지 말고 그대로 출력하세요. 
    """
)

# 파일 경로
file_path = r"techreader_data\LLM_TechLibrary.pdf"
# TechReader_gayoon\techreader_data\LLM_TechLibrary.pdf
# PDF → 파싱
parsed_docs = parser.load_data(file_path=file_path)

# LangChain Document 변환
docs = [doc.to_langchain_format() for doc in parsed_docs]

# Markdown 저장
file_root, _ = os.path.splitext(file_path)
output_file_path = file_root + "_parsed0903_gemini.md"

full_text = "\n\n".join([doc.page_content for doc in docs])

with open(output_file_path, "w", encoding="utf-8") as f:
    f.write(full_text)

print(f"✅ 파일 저장 완료: {output_file_path}")


Started parsing the file under job_id 899beb30-e77c-42c1-83aa-942499264a7b
✅ 파일 저장 완료: techreader_data\LLM_TechLibrary_parsed0903_gemini.md


In [7]:
import os
from llama_parse import LlamaParse

# PDF 파서 초기화
parser = LlamaParse(
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="openai-gpt4o",
    vendor_multimodal_api_key=os.environ["OPENAI_API_KEY"],
    result_type="markdown",
    parsing_mode="Unstructured",
    language="ko",
    parsing_instruction="""
    당신은 PDF 문서를 구조화된 Markdown으로 변환하는 파서입니다.
    
    가장 중요한 규칙:
    모든 텍스트는 가능한 모두 출력해주세요. 
    첫 페이지를 제외한 Tech Guide와 Tech Trend는 소제목이 아닙니다. 
    
    변환 규칙:
    0. 원문 텍스트는 가능한 한 모두 보존하세요. 표지도 그대로 출력하세요. 
    1. 문서의 '주요 제목'은 반드시 `# 제목` 형식으로 추출하세요.
       - 제목 바로 아래 줄에 '저자 | 소속'이 있으면 `Author: 이름 | 소속`으로 출력하세요.
    2. 본문 내의 소제목은 `## 소제목`으로 변환하세요. 
       - 소제목 바로 아래 줄에 문장들이 수록되므로, 참고하세요. 
       - 단, '# 1.' 같은 번호 형식, 영어 한 단어, 결론을 제외한 짧은 음절(예: 비추천 용도, 최적 용도, 주의점)은 소제목으로 간주하지 마세요. 
       - 첫 페이지를 제외하고는 Tech Guide와 Tech Trend는 소제목으로 정하지 마세요. 
       - 결론은 소제목이 될 수 있어요. 
    3. 제목/소제목 외의 일반 문단은 그냥 텍스트로 출력하세요. 특정 페이지에 일반 문단만 있어도 그대로 출력하세요. 
    4. 일반 문단은 그냥 텍스트로 출력하되 • 표시로 시작하는 것도 그대로 출력해주세요.   
    5. 모든 출력은 순수한 Markdown 형식으로 작성하세요. 불필요한 설명, 번역, 해설은 절대 추가하지 마세요. 텍스트를 요약하지 말고 그대로 출력하세요. 
    
    
    """ 
)


# 파일 경로
file_path = r"techreader_data\LLM_TechLibrary.pdf"
# TechReader_gayoon\techreader_data\LLM_TechLibrary.pdf
# PDF → 파싱
parsed_docs = parser.load_data(file_path=file_path)

# LangChain Document 변환
docs = [doc.to_langchain_format() for doc in parsed_docs]

# Markdown 저장
file_root, _ = os.path.splitext(file_path)
output_file_path = file_root + "_parsed0903_openai.md"

full_text = "\n\n".join([doc.page_content for doc in docs])

with open(output_file_path, "w", encoding="utf-8") as f:
    f.write(full_text)

print(f"✅ 파일 저장 완료: {output_file_path}")


Started parsing the file under job_id 2070d273-a279-4500-8205-52cddb41240f
✅ 파일 저장 완료: techreader_data\LLM_TechLibrary_parsed0903_openai.md


In [ ]:
# gemini.md 파일 사용하기로 결정 

In [12]:
# 06-MarkdownHeaderTextSplitter => ## 표시된 제목과 소제목 추출하는 파이프라인 

len(docs)


21

In [13]:
print(docs)

[Document(id='ab47db04-556d-4fe8-a0fd-9787d060d9fe', metadata={}, page_content='IT WORLD CIO\n\n# DEEP DIVE\n\n# “LLM 이후를 설계하다”\n## 생성형 AI의 과제와 대안 찾기\n\n**Tech Trend**\n- “아는 것만 아는” LLM, 오히려 혁신을 저해한다\n- LLM을 학습한 추출 모델, 작아도 위험은 동일\n\n**Tech Guide**\n- LLM 한계 극복을 위한 RAG의 역할과 최신 동향\n- 잊어버려야 할 것은 잊는 LLM이 필요한 시점\n- AI 코딩, LLM 혼합 전략이 답이다\n\n무단 전재 재배포 금지\n본 PDF 문서는 IDG Korea의 자산으로, 저작권법의 보호를 받습니다. IDG Korea의 허락 없이 PDF 문서를 온라인 사이트 등에 무단 게재, 전재하거나 유포할 수 없습니다.\n\n'), Document(id='b70f1fc4-d5a6-46ab-a35b-4861b8f38995', metadata={}, page_content='\n\nTech Trend\n\n# “아는 것만 아는” LLM, 오히려 혁신을 저해한다\n\n**Matt Asay | Infoworld**\n\n훈련 데이터에서 기존 기술이 가장 많은 비중을 차지하고 있는데, 대형 언어 모델(Large Language Model, LLM)이 더 나은 신기술을 추천할 이유는 무엇일까?\n\n작금의 소프트웨어 개발 시대는 조금 이상하다. 한쪽에서는 AI 기반 코딩 어시스턴트가 지금까지 굳건했던 IDE 시장을 뒤흔들고 있다. 레드몽크(RedMonk) 공동 설립자 제임스 거버너의 말처럼 “갑자기 편집기 시장에서 예상치 못한 혼란을 겪게 되었고”, “모든 것이 움직이고” “많은 혁신이 일어나는” 시대다. 아이러니하게도, 생성형 AI는 소프트웨어에도 혁신을 가져왔다. 다름 아닌 생성형 AI 코딩 어시스턴트가 계속해서 더 많이 추천하는 소프트웨어다.\n\nAWS 개발자 애드보킷 네이선 펙은

In [15]:
from langchain_text_splitters import MarkdownHeaderTextSplitter



In [18]:
import os
from langchain_text_splitters import MarkdownHeaderTextSplitter

# 1. 마크다운 파일 불러오기
with open("techreader_data/LLM_TechLibrary_parsed0903_gemini.md", "r", encoding="utf-8") as f:
    markdown_text = f.read()

# 2. 분할 기준 헤더 정의
headers_to_split_on = [
    ("#", "Header 1"),    # # 제목
    ("##", "Header 2"),   # ## 소제목
    ("###", "Header 3"),  # ### 하위 소제목
]

# 3. Splitter 초기화
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# 4. 문서 분할
md_header_splits = markdown_splitter.split_text(markdown_text)

# 5. 결과 출력
for i, doc in enumerate(md_header_splits[:10]):  # 앞에서 10개만 출력
    print(f"==== Chunk {i+1} ====")
    print(doc.page_content[:300])  # 앞부분 미리보기
    print("메타데이터:", doc.metadata)
    print()


==== Chunk 1 ====
IT WORLD CIO
메타데이터: {}

==== Chunk 2 ====
**Tech Trend**
- “아는 것만 아는” LLM, 오히려 혁신을 저해한다
- LLM을 학습한 추출 모델, 작아도 위험은 동일  
**Tech Guide**
- LLM 한계 극복을 위한 RAG의 역할과 최신 동향
- 잊어버려야 할 것은 잊는 LLM이 필요한 시점
- AI 코딩, LLM 혼합 전략이 답이다  
무단 전재 재배포 금지
본 PDF 문서는 IDG Korea의 자산으로, 저작권법의 보호를 받습니다. IDG Korea의 허락 없이 PDF 문서를 온라인 사이트 등에 무단 게재, 전재하거나 유포할 수 없습니다.  
Tech
메타데이터: {'Header 1': '“LLM 이후를 설계하다”', 'Header 2': '생성형 AI의 과제와 대안 찾기'}

==== Chunk 3 ====
**Matt Asay | Infoworld**  
훈련 데이터에서 기존 기술이 가장 많은 비중을 차지하고 있는데, 대형 언어 모델(Large Language Model, LLM)이 더 나은 신기술을 추천할 이유는 무엇일까?  
작금의 소프트웨어 개발 시대는 조금 이상하다. 한쪽에서는 AI 기반 코딩 어시스턴트가 지금까지 굳건했던 IDE 시장을 뒤흔들고 있다. 레드몽크(RedMonk) 공동 설립자 제임스 거버너의 말처럼 “갑자기 편집기 시장에서 예상치 못한 혼란을 겪게 되었고”, “모든 것이 움직이고” “많은 혁신이 일어나는” 시대다
메타데이터: {'Header 1': '“아는 것만 아는” LLM, 오히려 혁신을 저해한다'}

==== Chunk 4 ====
생성형 AI는 학습 데이터의 출처를 훼손하는 경향이 있다. 소프트웨어 개발 세계에서 챗GPT, 깃허브 코파일럿, 그리고 기타 대형 언어 모델은 개발자 생산성에 긍정적인 영향을 미쳤지만, 반대로 스택 오버플로와 같은 사이트에 매우 부정적인 영향을 미쳤다. 코파일럿을 사용할 수 있는데 굳이 스택 오버플로에 질문할 필요가

In [20]:
doc.metadata

{'Header 1': 'LLM을 학습한 추출 모델, 작아도 위험은 동일',
 'Header 2': '한층 간편해진 AI 공격',
 'Header 3': '정제 모델, 항상 보호되지는 않아'}

In [22]:
from collections import Counter

# 전체 chunk 개수
print("총 chunk 개수:", len(md_header_splits))

# 헤더별 카운트
header_counter = Counter()
for doc in md_header_splits:
    for key, value in doc.metadata.items():
        header_counter[key] += 1

print("헤더별 카운트:", header_counter)


총 chunk 개수: 31
헤더별 카운트: Counter({'Header 1': 30, 'Header 2': 18, 'Header 3': 14})


In [23]:
# 전체 chunk 내용 확인
for i, doc in enumerate(md_header_splits, start=1):
    print(f"==== Chunk {i} ====")
    print(doc.page_content.strip()[:500])  # 앞부분 500자만 미리보기 (너무 길면 잘림)
    print("메타데이터:", doc.metadata)
    print("=" * 50)
print(doc.page_content.strip())


==== Chunk 1 ====
IT WORLD CIO
메타데이터: {}
==== Chunk 2 ====
**Tech Trend**
- “아는 것만 아는” LLM, 오히려 혁신을 저해한다
- LLM을 학습한 추출 모델, 작아도 위험은 동일  
**Tech Guide**
- LLM 한계 극복을 위한 RAG의 역할과 최신 동향
- 잊어버려야 할 것은 잊는 LLM이 필요한 시점
- AI 코딩, LLM 혼합 전략이 답이다  
무단 전재 재배포 금지
본 PDF 문서는 IDG Korea의 자산으로, 저작권법의 보호를 받습니다. IDG Korea의 허락 없이 PDF 문서를 온라인 사이트 등에 무단 게재, 전재하거나 유포할 수 없습니다.  
Tech Trend
메타데이터: {'Header 1': '“LLM 이후를 설계하다”', 'Header 2': '생성형 AI의 과제와 대안 찾기'}
==== Chunk 3 ====
**Matt Asay | Infoworld**  
훈련 데이터에서 기존 기술이 가장 많은 비중을 차지하고 있는데, 대형 언어 모델(Large Language Model, LLM)이 더 나은 신기술을 추천할 이유는 무엇일까?  
작금의 소프트웨어 개발 시대는 조금 이상하다. 한쪽에서는 AI 기반 코딩 어시스턴트가 지금까지 굳건했던 IDE 시장을 뒤흔들고 있다. 레드몽크(RedMonk) 공동 설립자 제임스 거버너의 말처럼 “갑자기 편집기 시장에서 예상치 못한 혼란을 겪게 되었고”, “모든 것이 움직이고” “많은 혁신이 일어나는” 시대다. 아이러니하게도, 생성형 AI는 소프트웨어에도 혁신을 가져왔다. 다름 아닌 생성형 AI 코딩 어시스턴트가 계속해서 더 많이 추천하는 소프트웨어다.  
AWS 개발자 애드보킷 네이선 펙은 “AI 코딩 어시스턴트의 마법 같은 기능 이면에는 냉혹한 진실이 있다”라며 “훈련받은 데이터만큼만 능력이 발휘되고, 그 때문에 새로운 프레임워크가 제한된다”라고 지적했다.
메타데이터: {'Header 1': '“아는 것만 아는” LLM, 오히려 혁신을

In [26]:
import csv

with open("techreader_data/chunks_output.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["Chunk No", "Metadata", "Content"])
    for i, doc in enumerate(md_header_splits, start=1):
        writer.writerow([i, doc.metadata, doc.page_content.strip()])


In [27]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
import os

# 1. OpenAI Embedding 초기화
embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])

# 2. md_header_splits (문서 리스트)를 벡터화 후 저장
vectorstore = FAISS.from_documents(md_header_splits, embeddings)

# 3. 저장
vectorstore.save_local("faiss_index")

print("✅ 벡터스토어 저장 완료: faiss_index 폴더 생성됨")


✅ 벡터스토어 저장 완료: faiss_index 폴더 생성됨


In [28]:
# 저장된 FAISS 인덱스 불러오기
new_vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

# 검색 예시
query = "RAG 최신 동향"
docs = new_vectorstore.similarity_search(query, k=3)

for d in docs:
    print("📌", d.page_content[:300])
    print("메타데이터:", d.metadata)
    print("="*50)


📌 RAG 성능을 더욱 향상시키려면 임베딩 모델을 미세 조정해 검색된 정보의 관련성을 높일 수 있다. 예를 들어, 회사 고객 지원 질의 데이터를 활용할 때 검색된 정보의 품질이 최대 41% 향상될 수 있다. 참고로 구글은 평균 12% 개선된다고 보고했다.  
혹은 다양한 변형 RAG 아키텍처를 적용해 LLM 기반 애플리케이션의 성능을 개선하는 방법도 있다. 현재 수십 가지의 변형 방식이 존재하지만, 대표적인 몇 가지는 다음과 같다.  
* **검색 및 재순위(Retrieve and Re-rank)** : 검색된 정보를 더욱 정교하게 선
메타데이터: {'Header 1': 'LLM 한계 극복을 위한 RAG의 역할과 최신 동향', 'Header 2': 'RAG 개선하기'}
📌 이런 문제를 해결하는 한 가지 방법이 RAG다. RAG는 사용자가 인터넷이나 문서를 검색한 후, 검색 결과를 바탕으로 언어 모델에 답변을 요청하는 두 단계를 결합한다. 검색 결과가 언어 모델의 컨텍스트 한계를 초과하는 문제를 우회할 수 있는 방법이다.  
RAG의 첫 번째 단계는 쿼리할 소스 정보를 고밀도, 고차원 형태로 벡터화하는 것이다. 일반적으로 임베딩 벡터를 생성한 후 이를 벡터 데이터베이스에 저장해 고차원 공간에서 밀집된 형태로 변환한다.  
그런 다음 쿼리 자체를 벡터화하고, FAISS, 쿼드런트(Qdrant) 또는 기타
메타데이터: {'Header 1': 'LLM 한계 극복을 위한 RAG의 역할과 최신 동향', 'Header 2': '해결책 : LLM과 사실의 그라운딩'}
📌 **Martin Heller | Infoworld**  
검색 증강 생성(Retrieval-Augmented Generation, RAG)는 LLM을 특정 데이터 소스로 그라운딩(grounding, 모델을 새 데이터에 연결하는 것)하는 기법으로, 일반적으로 모델 초기의 학습 데이터에 포함되지 않은 정보를 활용한다. RAG는 세 단계로 구성된다. 먼저 지정된 소스에서 관련 정보를 검색한 후, 검색된 데이터를 

In [29]:
query = "교사 모델과 학생 모델의 관계에서 발생하는 보안 위험은 무엇인가요?"
docs = new_vectorstore.similarity_search(query, k=3)

for d in docs:
    print("📌", d.page_content[:300])
    print("메타데이터:", d.metadata)
    print("="*50)


📌 추출된 모델은 훈련 데이터에 내재된 보안 위험을 포함해 원래 모델의 행동 상당 부분을 그대로 물려받는다. 지적 재산권 도용, 개인 정보 유출, 모델 반전 공격 등의 위험을 그대로 떠안는 것이다.  
브라우클러는 “일반적으로 모델 추출은 원래 더 큰 교사 모델이 소비한 훈련 데이터와 교사 모델의 유효한 결과(결과의 확률 분포 등) 예측을 사용한다. 결과적으로 학생 모델이 훈련 세트의 민감한 데이터를 포함해 교사 모델과 동일한 행동을 많이 기억할 기회가 생긴다”라고 설명했다.  
4  
Tech Trend  
교사 모델의 보안 취약점은
메타데이터: {'Header 1': 'LLM을 학습한 추출 모델, 작아도 위험은 동일', 'Header 2': '교사 모델 부담을 떠맡은 학생 모델'}
📌 학생 모델은 스스로 맥락을 이해하지 않고 교사 모델의 사전 학습된 결론에 크게 의존한다. 이런 제한이 모델 환각으로 이어질지에 대해서는 전문가 사이에서 많은 논란이 있다.  
브라우클러는 훈련 방식에 관계없이 학생 모델의 효율성은 곧 교사 모델의 효율성과 관련이 있다고 생각한다. 즉, 교사 모델에서 환각이 없으면 학생 모델에도 없을 가능성이 높다는 의미다.  
가트너의 애널리스트 아룬 찬드라세카란도 대부분 동의하지만, 학생 모델은 규모와 목적에 있어 새로운 환각을 보이는 문제점이 있다고 주장했다.  
찬드라세카란은 “추출 자체로 바
메타데이터: {'Header 1': 'LLM을 학습한 추출 모델, 작아도 위험은 동일', 'Header 2': '교사 모델 부담을 떠맡은 학생 모델', 'Header 3': '지혜를 전수받지만 취약성은 커져'}
📌 추출의 또 다른 단점은 해석 가능성이다. LLM은 보안 팀이 근본 원인 조사에서 광범위한 로그와 복잡한 의사 결정 경로를 분석할 수 있다는 이점이 있다. 그러나 정제된 모델은 이런 세분성이 부족해 취약점을 진단하거나 보안 사고를 추적하기가 더 어렵다.  
찬드라세카란은 “사고 대응의 맥락에서 학생 모델의 세부 로그와 매개 변수가 부족하면

In [ ]:
# index.faiss 
faiss에서 실제 벡터를 저장하는 파일
모든 청크 텍스트가 임베딩된 후, 그 임베딩 벡터를 효율적으로 저장 검색하기 위해 faiss 라이브러리 포맷으로 저장된다. 
벡터 공간 좌표 

# index.pickle
벡터와 연결된 메타데이터 + 원문 텍스트를 저장하는 파일 
index.faiss에 있는 숫자 벡터와 index.pkl에 있는 텍스트 메타데이터를 매칭시켜 
이 벡터가 어떤 문서 조각인지 알 수 있게 해준다. 

index.faiss → 빠른 검색을 위한 벡터 인덱스 데이터
index.pkl → 벡터와 연결된 텍스트 & 메타데이터 저장소

# 두 파일이 세트로 있어야, FAISS.load_local()로 다시 불러 들일 수 있따. 

In [ ]:
# 리트리버 생성 전, 예상 질문 생성 - LLM 활용 


In [32]:
import os
import google.generativeai as genai

# Google AI Studio에서 발급받은 API 키 설정
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# Gemini 모델 초기화 (최신은 gemini-1.5-pro 또는 gemini-1.5-flash)
model = genai.GenerativeModel("gemini-2.5-pro")

def generate_questions(header1, header2=None):
    topic = header1 if header2 is None else f"{header1} - {header2}"
    prompt = f"""
    너는 AI 최신 기술 동향을 분석해 사내 엔지니어와 연구원이 빠르게 이해할 수 있도록
    핵심 쟁점을 질문 형태로 정리하는 역할이다.
    지금 다루는 문서는 Tech Library Top1에 선정된 21페이지짜리 리포트이며, 재직자 전용이다.
    너의 목표는 주제와 관련된 기술적 쟁점과 연구 방향을 드러내는 예상 질문을 생성하는 것이다.

    [요구사항]
    - 주제: {topic}
    - 예상 질문은 총 5개를 만들어라.
    - 질문은 학생용 단순 이해 차원이 아니라, 엔지니어/연구자가
      토론·실험·설계 단계에서 실제로 고민할 만한 '기술적 질문'이어야 한다.
    - 질문은 명확하고 구체적으로 작성하라.
    """

    response = model.generate_content(prompt)
    return response.text.strip().split("\n")


In [33]:
questions_dict = {}

for doc in md_header_splits:
    h1 = doc.metadata.get("Header 1")
    h2 = doc.metadata.get("Header 2")

    if (h1, h2) not in questions_dict:
        q_list = generate_questions(h1, h2)
        questions_dict[(h1, h2)] = q_list

# 확인
for (h1, h2), q_list in questions_dict.items():
    print(f"\n📌 {h1} > {h2 if h2 else ''}")
    for q in q_list:
        print(" -", q)



📌 None > 
 - 알겠습니다. AI 최신 기술 동향 분석 리포트를 바탕으로, 사내 엔지니어와 연구원의 심도 있는 논의를 유도할 핵심 기술 쟁점 질문 5개를 생성하겠습니다.
 - 
 - 제공된 문서의 구체적인 주제가 없어, 현재 AI 분야에서 가장 활발히 논의되는 주제 중 하나인 **'대규모 멀티모달 모델을 위한 Mixture-of-Experts (MoE) 아키텍처 최적화'**를 가정하고 질문을 생성했습니다.
 - 
 - ---
 - 
 - ### **[예상 질문] 대규모 멀티모달 MoE 모델의 기술적 쟁점**
 - 
 - 1.  **(라우팅 및 부하 분산)** 현재 MoE 모델에서 사용되는 Top-k 게이팅 기반 라우팅 전략이 특정 유형의 토큰(예: 코드, 비전 데이터)에 편중되는 현상을 어떻게 완화할 수 있을까요? 토큰의 모달리티(modality) 특성을 반영한 동적 라우팅(dynamic routing) 알고리즘을 설계한다면, 전문가 간 로드 밸런싱과 전문가 특화(specialization) 사이의 최적 트레이드오프 지점은 어디일까요?
 - 
 - 2.  **(시스템 및 추론 최적화)** MoE 아키텍처의 파라미터 수는 방대하지만 추론 시 활성화되는 파라미터는 일부입니다. 서비스 배포 관점에서, 비활성 전문가(inactive experts)를 효율적으로 관리하기 위한 메모리 오프로딩(offloading) 또는 스와핑(swapping) 전략은 무엇이 있을까요? 특히, 실시간 추론 지연 시간(inference latency)에 미치는 영향을 최소화하면서 GPU 메모리 사용량을 최적화할 수 있는 시스템 레벨의 설계 방안은 무엇일까요?
 - 
 - 3.  **(멀티모달 융합 아키텍처)** 멀티모달 MoE에서, 텍스트와 이미지 임베딩을 동일한 전문가 네트워크 집합으로 라우팅하는 것이 최선일까요, 아니면 모달리티별 전용 전문가(modality-specific experts) 그룹을 두는 것이 더 효과적일까요? 후자의 경우, 서로 다른 모달리티의 정보를 융합(fu

In [ ]:
len(q) 

279

In [35]:
# 총 예상 질문 개수
total_questions = sum(len(q_list) for q_list in questions_dict.values())
print("총 예상 질문 개수:", total_questions)

# 카테고리별 개수 출력
for (h1, h2), q_list in questions_dict.items():
    print(f"{h1} > {h2 if h2 else ''} : {len(q_list)}개")


총 예상 질문 개수: 399
None >  : 17개
“LLM 이후를 설계하다” > 생성형 AI의 과제와 대안 찾기 : 26개
“아는 것만 아는” LLM, 오히려 혁신을 저해한다 >  : 27개
“아는 것만 아는” LLM, 오히려 혁신을 저해한다 > 새 기술을 제안하지 않는 LLM : 22개
LLM을 학습한 추출 모델, 작아도 위험은 동일 >  : 23개
LLM을 학습한 추출 모델, 작아도 위험은 동일 > 교사 모델 부담을 떠맡은 학생 모델 : 18개
LLM을 학습한 추출 모델, 작아도 위험은 동일 > 한층 간편해진 AI 공격 : 13개
LLM 한계 극복을 위한 RAG의 역할과 최신 동향 >  : 25개
LLM 한계 극복을 위한 RAG의 역할과 최신 동향 > LLM의 문제 : 환각, 제한적인 컨텍스트 : 25개
LLM 한계 극복을 위한 RAG의 역할과 최신 동향 > 해결책 : LLM과 사실의 그라운딩 : 27개
LLM 한계 극복을 위한 RAG의 역할과 최신 동향 > RAG 개선하기 : 25개
잊어버려야 할 것은 잊는 LLM이 필요한 시점 >  : 25개
잊어버려야 할 것은 잊는 LLM이 필요한 시점 > LLM 애플리케이션에서 메모리가 작동하는 방식 : 22개
잊어버려야 할 것은 잊는 LLM이 필요한 시점 > LLM에서 상태가 유지되지 않는 이유 : 20개
잊어버려야 할 것은 잊는 LLM이 필요한 시점 > Tech Guide : 32개
AI 코딩, LLM 혼합 전략이 답이다 >  : 25개
AI 코딩, LLM 혼합 전략이 답이다 > Tech Guide : 27개


In [ ]:
# 위의 예상 질문 : md_header_splits에서 뽑은 헤더 텍스트를 기반으로 예상 질문 생성했따. 
# generate_questions(h1, h2)는 실제 벡터스토어(Faiss)에 저장된 내용을 직접 참조하지는 않는다. 

# 그래서 실제 보고서 내용 기반으로 헤더 텍스트와 연결한 예상 질문 리스트 필요하다 


In [37]:
# 새 버전 (헤더 + 본문 기반)
def generate_questions_from_content(header1, header2=None, content=None):
    topic = header1 if header2 is None else f"{header1} - {header2}"
    prompt = f"""
    너는 AI 최신 기술 동향 보고서를 분석해 사내 엔지니어와 연구원이 빠르게 이해할 수 있도록
    핵심 쟁점을 질문 형태로 정리하는 역할이다.
    지금 다루는 문서는 Tech Library Top1에 선정된 21페이지짜리 리포트이며, 재직자 전용이다.

    [입력 정보]
    - 주제: {topic}
    - 본문 내용: {content[:1500] if content else ""}
    
    [요구사항]
    - 위 본문 내용과 주제를 바탕으로 예상 질문을 총 5개 만들어라.
    - 질문은 엔지니어/연구자가 토론·실험·설계 단계에서 실제로 고민할 만한
      '기술적 질문'이어야 한다.
    - 질문은 명확하고 구체적으로 작성하라.
    """
    response = model.generate_content(prompt)
    return response.text.strip().split("\n")


In [38]:
# 보고서 본문 기반 질문 리스트
content_questions_dict = {}

for doc in md_header_splits:
    h1 = doc.metadata.get("Header 1")
    h2 = doc.metadata.get("Header 2")
    content = doc.page_content

    if (h1, h2) not in content_questions_dict:
        q_list = generate_questions_from_content(h1, h2, content)
        content_questions_dict[(h1, h2)] = q_list

# 확인
for (h1, h2), q_list in content_questions_dict.items():
    print(f"\n📌 {h1} > {h2 if h2 else ''}")
    for q in q_list:
        print(" -", q)



📌 None > 
 - 알겠습니다. AI 최신 기술 동향 보고서를 바탕으로, 사내 엔지니어와 연구원의 기술적 토론과 실험 설계를 유도할 수 있는 핵심 질문 5개를 도출해 보겠습니다.
 - 
 - 입력된 정보(주제: None, 본문: IT WORLD CIO)를 고려할 때, 현재 CIO들이 가장 주목하는 기술, 즉 **'엔터프라이즈 생성형 AI(Generative AI) 도입 및 운영 전략'**이 핵심 주제일 가능성이 매우 높습니다. 이 주제를 바탕으로 다음과 같은 기술적 쟁점을 질문 형태로 정리했습니다.
 - 
 - ---
 - 
 - ### **[기술 동향 보고서 핵심 쟁점] 엔지니어/연구원을 위한 예상 질문 5가지**
 - 
 - **1. [RAG 아키텍처] 사내 데이터베이스와의 실시간 연동을 위한 최적의 RAG(Retrieval-Augmented Generation) 파이프라인 설계 방안은 무엇인가?**
 - - 실시간으로 변경되는 벡터 데이터베이스(Vector DB)의 인덱싱 지연을 최소화하고, LLM(거대 언어 모델)이 항상 최신 정보를 참조하도록 보장하려면 어떤 기술 스택(e.g., CDC, Incremental Indexing) 조합이 가장 효과적일까요? 정확성과 응답 속도 간의 트레이드오프는 어떻게 관리해야 할까요?
 - 
 - **2. [LLM 서빙 최적화] 자체 호스팅(On-premise/VPC) LLM의 추론(Inference) 비용을 현재의 50% 수준으로 절감하기 위한 구체적인 최적화 전략은 무엇인가?**
 - - 현재 우리가 사용하는 GPU 리소스 대비 처리량(Throughput)을 극대화하기 위해, 모델 경량화(Quantization, Pruning)와 서빙 프레임워크(e.g., vLLM, TGI) 도입 중 어떤 것을 우선적으로 테스트해야 할까요? 배치(Batch) 처리 크기와 응답 지연 시간(Latency)의 허용 기준은 어떻게 설정해야 할까요?
 - 
 - **3. [AI 보안 및 가드레일] 생성형 AI 기반 서비스에서 발생 가능

In [39]:
# 전체 보고서 내용 기반 질문 개수 세기 
# 총 예상 질문 개수
total_questions = sum(len(q_list) for q_list in content_questions_dict.values())
print("총 예상 질문 개수:", total_questions)

# 주제/소제목별 개수 확인하기 
for (h1, h2), q_list in content_questions_dict.items():
    print(f"{h1} > {h2 if h2 else ''} : {len(q_list)}개") 

총 예상 질문 개수: 298
None >  : 22개
“LLM 이후를 설계하다” > 생성형 AI의 과제와 대안 찾기 : 15개
“아는 것만 아는” LLM, 오히려 혁신을 저해한다 >  : 20개
“아는 것만 아는” LLM, 오히려 혁신을 저해한다 > 새 기술을 제안하지 않는 LLM : 15개
LLM을 학습한 추출 모델, 작아도 위험은 동일 >  : 13개
LLM을 학습한 추출 모델, 작아도 위험은 동일 > 교사 모델 부담을 떠맡은 학생 모델 : 13개
LLM을 학습한 추출 모델, 작아도 위험은 동일 > 한층 간편해진 AI 공격 : 20개
LLM 한계 극복을 위한 RAG의 역할과 최신 동향 >  : 20개
LLM 한계 극복을 위한 RAG의 역할과 최신 동향 > LLM의 문제 : 환각, 제한적인 컨텍스트 : 18개
LLM 한계 극복을 위한 RAG의 역할과 최신 동향 > 해결책 : LLM과 사실의 그라운딩 : 25개
LLM 한계 극복을 위한 RAG의 역할과 최신 동향 > RAG 개선하기 : 20개
잊어버려야 할 것은 잊는 LLM이 필요한 시점 >  : 22개
잊어버려야 할 것은 잊는 LLM이 필요한 시점 > LLM 애플리케이션에서 메모리가 작동하는 방식 : 17개
잊어버려야 할 것은 잊는 LLM이 필요한 시점 > LLM에서 상태가 유지되지 않는 이유 : 13개
잊어버려야 할 것은 잊는 LLM이 필요한 시점 > Tech Guide : 15개
AI 코딩, LLM 혼합 전략이 답이다 >  : 15개
AI 코딩, LLM 혼합 전략이 답이다 > Tech Guide : 15개


# 생성한 예상 질문 리스트 - 파일에 각각 저장 (header 기준 질문 생성, header와 content 기준 질문 생성)

In [ ]:
# 각 변수가 너무 난잡하게 들어가있어서 필터링 필요 
실제 질문은 물음표로 끝나므로 각 질문만 뽑아서 Header와 연결해서 CSV/JSON에 저장

In [41]:
import csv
import re

def clean_question(q: str) -> str:
    """
    질문 문자열에서 불필요한 접두사 제거
    """
    q = re.sub(r"^\d+[\.\)]\s*", "", q).strip()   # 숫자 목록 제거 (예: "1. " , "2) ")
    q = re.sub(r"^[-•]\s*", "", q).strip()        # bullet 제거
    q = re.sub(r"^\*\*|\*\*$", "", q).strip()     # bold 제거
    return q

# 1. 헤더 기반 질문 저장
with open("techreader_data/header_based_questions_clean.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["Header 1", "Header 2", "Question"])
    for (h1, h2), q_list in questions_dict.items():  
        for q in q_list:
            q_clean = clean_question(q)
            if q_clean.endswith("?"):  # 진짜 질문만 저장
                writer.writerow([h1, h2 if h2 else "", q_clean])

print("✅ header_based_questions_clean.csv 저장 완료")

# 2. 본문 기반 질문 저장
with open("techreader_data/content_based_questions_clean.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["Header 1", "Header 2", "Question"])
    for (h1, h2), q_list in content_questions_dict.items():  
        for q in q_list:
            q_clean = clean_question(q)
            if q_clean.endswith("?"):  # 진짜 질문만 저장
                writer.writerow([h1, h2 if h2 else "", q_clean])

print("✅ content_based_questions_clean.csv 저장 완료")


✅ header_based_questions_clean.csv 저장 완료
✅ content_based_questions_clean.csv 저장 완료


In [ ]:
# 각 csv 파일 확인해본 결과 개수 감소 
content_questions_dict 개수는 298개였는데 88개로 되었고 questions_dict 개수는 399개였는데 87개가 되었어
물음표로 끝나지 않는 줄은 다 버려진 거예요.

# 답변도 같이 생성해서 만들어두기 

In [42]:
import csv

def load_questions(csv_path):
    questions = []
    with open(csv_path, "r", encoding="utf-8-sig") as f:
        reader = csv.DictReader(f)
        for row in reader:
            questions.append({
                "Header 1": row["Header 1"],
                "Header 2": row["Header 2"],
                "Question": row["Question"]
            })
    return questions

questions = load_questions("techreader_data/content_based_questions_clean.csv")
print(f"총 {len(questions)}개 질문 불러옴")


총 87개 질문 불러옴


In [ ]:
# 답변은 3문단 정도, 서론 - 본론 - 결론 식으로 a4 절반 혹은 1쪽 분량 (500~800자) 
답변 마무리를 하지 않는 경우가 있어 max_output_tokens 늘리기 
gemini api에서는 1024가 최대 토큰이다. 그래야 문장을 끝까지 출력한다. 

In [76]:
import google.generativeai as genai
import os, re

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-2.5-pro")

def clean_answer_text(text: str) -> str:
    # 불필요한 멘트 제거
    text = re.sub(r"(네, 알겠습니다.*|물론입니다.*|다음은.*|아래와 같이.*)", "", text)
    text = re.sub(r"[*#]{2,}", "", text)  # ###, *** 제거
    # 문장 끝의 ... → .
    text = re.sub(r"\.{2,}", ".", text.strip())
    return text.strip()

def generate_answer(question, header1, header2, content):
    prompt = f"""
    너는 AI 최신 기술 리포트를 분석하는 LLM 엔지니어다.
    문서 주제: {header1} - {header2 if header2 else ""}
    본문 내용 (발췌): {content[:3000] if content else ""}

    [요구사항]
    - 아래 질문에 대해 보고서 본문을 근거로 심층적인 답변을 작성하라.
    - 답변은 연구 보고서 스타일로 작성하며, 3~4문단으로 구성하라.
    - 전체 분량은 800~1000자 내외가 되도록 하라.
    - 각 문단은 완결된 문장으로 끝내라.
    - 서론(질문의 중요성), 본론(기술적 근거·세부 분석), 결론(핵심 요약과 시사점) 구조를 따르라.
    - 마지막 문장은 반드시 마침표 하나(.)로 끝내라. 불필요한 ... 은 쓰지 말라.
    - 출력은 반드시 '답변: ' 형식으로 하라.
    - 최종 답변은 반드시 마침표 하나(.)로 끝내라.
    - '...' 나 불필요한 반복 마침표는 절대 사용하지 말라.


    질문: {question}
    """

    try:
        response = model.generate_content(
            prompt,
            generation_config={"max_output_tokens": 9096, "temperature": 0.7}
        )

        if response.candidates and response.candidates[0].content.parts:
            answer = response.candidates[0].content.parts[0].text.strip()
        else:
            answer = "[⚠️ 답변 없음: 토큰 한도 초과 또는 안전 필터 차단]"

    except Exception as e:
        answer = f"[⚠️ 에러 발생: {str(e)}]"

    # 후처리: 결론 보강
    if not answer.endswith(("이다.", "있다.", "할 수 있다.")):
        try:
            fix_prompt = f"""
            다음 답변이 결론 없이 끝났습니다. 
            불필요한 멘트(예: '네, 알겠습니다', '물론입니다', '다음은', '### 결론', '***')는 쓰지 말고, 
            보고서 스타일의 결론 문단(3~4문장)을 작성하세요. 마지막 문장은 반드시 마침표 하나(.)로 끝내라.

            불완전 답변: {answer}
            """
            fix_response = model.generate_content(fix_prompt)
            if fix_response.candidates and fix_response.candidates[0].content.parts:
                answer += "\n\n" + fix_response.candidates[0].content.parts[0].text.strip()
        except:
            pass

    # 마지막 정리 (멘트/###/*** 제거, ... → .)
    return clean_answer_text(answer) or ""


In [77]:
for i, q in enumerate(questions[:3]):
    h1, h2, question = q["Header 1"], q["Header 2"], q["Question"]

    # md_header_splits에서 해당 Header 매칭되는 본문 가져오기
    content = ""
    for doc in md_header_splits:
        if doc.metadata.get("Header 1") == h1 and doc.metadata.get("Header 2") == h2:
            content = doc.page_content
            break

    answer = generate_answer(question, h1, h2, content)
    print(f"\nQ{i+1}: {question}")  
    print(f"A: {answer[:]}...\n")  # 앞부분 미리보기



Q1: 1. [RAG 아키텍처] 사내 데이터베이스와의 실시간 연동을 위한 최적의 RAG(Retrieval-Augmented Generation) 파이프라인 설계 방안은 무엇인가?
A: 답변: 사내 데이터베이스와의 실시간 연동을 위한 최적의 RAG(Retrieval-Augmented Generation) 파이프라인 설계는 기업의 동적 데이터를 LLM(거대 언어 모델)에 안전하고 신속하게 통합하는 핵심 과제입니다. LLM이 가진 일반 지식의 한계를 극복하고, 시시각각 변하는 내부 데이터에 기반한 정확하고 신뢰도 높은 답변을 생성하기 위해서는 단순한 정보 검색을 넘어선 고도화된 아키텍처가 요구됩니다. 이는 단순히 기술적 구현을 넘어, 데이터의 최신성, 검색 정확도, 시스템 응답 속도, 그리고 보안이라는 네 가지 핵심 요소를 균형 있게 고려하는 종합적인 접근을 필요로 하므로, 파이프라인 각 단계에 대한 심층적인 설계 전략이 매우 중요합니다.

최적의 파이프라인 설계를 위한 기술적 핵심은 ‘실시간 데이터 동기화’와 ‘하이브리드 검색(Hybrid Search)’의 유기적 결합에 있습니다. 우선, 데이터베이스의 변경 사항을 실시간으로 벡터 인덱스에 반영하기 위해 CDC(Change Data Capture) 기술이나 메시지 큐(Message Queue) 기반의 이벤트-드리븐(Event-Driven) 아키텍처 도입이 필수적입니다. 데이터가 생성, 수정, 삭제될 때마다 관련 이벤트를 트리거하여 임베딩 및 인덱싱 파이프라인을 자동으로 실행시키는 방식은 데이터 정합성을 극대화합니다. 다음으로, 검색 단계에서는 의미적 유사도를 찾는 벡터 검색과 키워드, 제품 코드 등 정형화된 정보를 정확히 찾는 텍스트 검색(예: BM25)을 결합한 하이브리드 검색을 적용해야 합니다. 이는 사용자의 복합적인 질의 의도를 보다 정밀하게 파악하고, 의미적으로는 유사하지만 핵심 키워드가 누락된 검색 실패 사례를 방지하여 검색(Retrieval)의 정확도를 비약적으로 향상시킵니다. 또한, 검색된 문서들의 

In [79]:
for i, q in enumerate(questions):  # 전체 다 돌림
    h1, h2, question = q["Header 1"], q["Header 2"], q["Question"]

    # md_header_splits에서 헤더 매칭해서 본문 불러오기
    content = ""
    for doc in md_header_splits:
        if doc.metadata.get("Header 1") == h1 and doc.metadata.get("Header 2") == h2:
            content = doc.page_content
            break

    answer = generate_answer(question, h1, h2, content)
    q["Answer"] = answer  # 답변 추가

    print(f"\nQ{i+1}/{len(questions)}: {question}")
    print(f"A: {answer[:]}...")  # 앞부분만 미리보기 
    
    
    
import csv

output_path = "techreader_data/content_based_questions_with_answers.csv"

with open(output_path, "w", newline="", encoding="utf-8-sig") as f:
    fieldnames = ["Header 1", "Header 2", "Question", "Answer"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

    for q in questions:
        writer.writerow({
            "Header 1": q["Header 1"],
            "Header 2": q["Header 2"],
            "Question": q["Question"],
            "Answer": q.get("Answer", "")
        })

print(f"✅ 최종 저장 완료: {output_path}")




Q1/87: 1. [RAG 아키텍처] 사내 데이터베이스와의 실시간 연동을 위한 최적의 RAG(Retrieval-Augmented Generation) 파이프라인 설계 방안은 무엇인가?
A: 답변:
기업 환경에서 대규모 언어 모델(LLM)의 활용 가치를 극대화하기 위해 사내 데이터베이스와의 실시간 연동은 핵심적인 과제로 부상하고 있습니다. 정적인 문서 기반의 전통적인 RAG(Retrieval-Augmented Generation) 방식은 실시간으로 변동하는 재고, 고객 정보, 재무 데이터 등을 정확히 반영하지 못하는 명백한 한계를 가집니다. 따라서 LLM이 최신 데이터를 기반으로 신뢰도 높은 답변을 생성하게 하려면, 데이터의 동적 특성을 파이프라인 설계 단계부터 고려하는 고도화된 접근법이 필수적이며, 이는 기업의 의사결정 지원 및 업무 자동화 시스템 구축에 있어 매우 중요한 요소입니다.

본 보고서에서 분석한 최적의 RAG 파이프라인은 ‘쿼리 분석 기반 적응형 검색(Query-Aware Adaptive Retrieval)’ 아키텍처에 해당합니다. 이 모델은 사용자 질의의 의도를 먼저 파악하는 ‘쿼리 라우터(Query Router)’를 파이프라인 전면에 배치하는 것이 핵심입니다. 쿼리 라우터는 질의가 정적인 정보(예: 회사 정책 문서)를 요구하는지, 아니면 동적인 실시간 데이터(예: 현재 재고 수량)를 요구하는지를 판단합니다. 질의가 동적 데이터를 필요로 할 경우, 파이프라인은 벡터 검색 대신 Text-to-SQL과 같은 모델을 호출하여 사내 데이터베이스에 직접 SQL 쿼리를 실행하고 그 결과를 즉시 가져옵니다. 반면, 정적인 정보가 필요할 경우에는 기존 방식대로 벡터 데이터베이스에서 관련 문서를 검색하며, 두 가지 정보를 모두 요구하는 복합 질의에 대해서는 병렬적으로 두 경로를 모두 실행한 후 결과를 통합하여 LLM에 전달합니다. 이 방식은 불필요한 벡터 검색을 줄여 응답 속도를 개선하고, 데이터베이스에 직접 접근함으로써 정보의 최신성과 정확성을 극대화하는 효

In [80]:
import csv

def load_questions(csv_path):
    questions = []
    with open(csv_path, "r", encoding="utf-8-sig") as f:
        reader = csv.DictReader(f)
        for row in reader:
            questions.append({
                "Header 1": row["Header 1"],
                "Header 2": row["Header 2"],
                "Question": row["Question"]
            })
    return questions

questions = load_questions("techreader_data/header_based_questions_clean.csv")
print(f"총 {len(questions)}개 질문 불러옴")


총 86개 질문 불러옴


In [81]:
import google.generativeai as genai
import os, re

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-2.5-pro")

def clean_answer_text(text: str) -> str:
    # 불필요한 멘트 제거
    text = re.sub(r"(네, 알겠습니다.*|물론입니다.*|다음은.*|아래와 같이.*)", "", text)
    text = re.sub(r"[*#]{2,}", "", text)  # ###, *** 제거
    # 문장 끝의 ... → .
    text = re.sub(r"\.{2,}", ".", text.strip())
    return text.strip()

def generate_answer(question, header1, header2, content):
    prompt = f"""
    너는 AI 최신 기술 리포트를 분석하는 LLM 엔지니어다.
    문서 주제: {header1} - {header2 if header2 else ""}
    본문 내용 (발췌): {content[:3000] if content else ""}

    [요구사항]
    - 아래 질문에 대해 보고서 본문을 근거로 심층적인 답변을 작성하라.
    - 답변은 연구 보고서 스타일로 작성하며, 3~4문단으로 구성하라.
    - 전체 분량은 800~1000자 내외가 되도록 하라.
    - 각 문단은 완결된 문장으로 끝내라.
    - 서론(질문의 중요성), 본론(기술적 근거·세부 분석), 결론(핵심 요약과 시사점) 구조를 따르라.
    - 마지막 문장은 반드시 마침표 하나(.)로 끝내라. 불필요한 ... 은 쓰지 말라.
    - 출력은 반드시 '답변: ' 형식으로 하라.
    - 최종 답변은 반드시 마침표 하나(.)로 끝내라.
    - '...' 나 불필요한 반복 마침표는 절대 사용하지 말라.


    질문: {question}
    """

    try:
        response = model.generate_content(
            prompt,
            generation_config={"max_output_tokens": 9096, "temperature": 0.7}
        )

        if response.candidates and response.candidates[0].content.parts:
            answer = response.candidates[0].content.parts[0].text.strip()
        else:
            answer = "[⚠️ 답변 없음: 토큰 한도 초과 또는 안전 필터 차단]"

    except Exception as e:
        answer = f"[⚠️ 에러 발생: {str(e)}]"

    # 후처리: 결론 보강
    if not answer.endswith(("이다.", "있다.", "할 수 있다.")):
        try:
            fix_prompt = f"""
            다음 답변이 결론 없이 끝났습니다. 
            불필요한 멘트(예: '네, 알겠습니다', '물론입니다', '다음은', '### 결론', '***')는 쓰지 말고, 
            보고서 스타일의 결론 문단(3~4문장)을 작성하세요. 마지막 문장은 반드시 마침표 하나(.)로 끝내라.

            불완전 답변: {answer}
            """
            fix_response = model.generate_content(fix_prompt)
            if fix_response.candidates and fix_response.candidates[0].content.parts:
                answer += "\n\n" + fix_response.candidates[0].content.parts[0].text.strip()
        except:
            pass

    # 마지막 정리 (멘트/###/*** 제거, ... → .)
    return clean_answer_text(answer) or ""


In [82]:
for i, q in enumerate(questions[:3]):  # 전체 다 돌림
    h1, h2, question = q["Header 1"], q["Header 2"], q["Question"]

    # md_header_splits에서 헤더 매칭해서 본문 불러오기
    content = ""
    for doc in md_header_splits:
        if doc.metadata.get("Header 1") == h1 and doc.metadata.get("Header 2") == h2:
            content = doc.page_content
            break

    answer = generate_answer(question, h1, h2, content)
    q["Answer"] = answer  # 답변 추가

    print(f"\nQ{i+1}/{len(questions)}: {question}")
    print(f"A: {answer[:]}...")  # 앞부분만 미리보기 
    
    


Q1/86: (라우팅 및 부하 분산)** 현재 MoE 모델에서 사용되는 Top-k 게이팅 기반 라우팅 전략이 특정 유형의 토큰(예: 코드, 비전 데이터)에 편중되는 현상을 어떻게 완화할 수 있을까요? 토큰의 모달리티(modality) 특성을 반영한 동적 라우팅(dynamic routing) 알고리즘을 설계한다면, 전문가 간 로드 밸런싱과 전문가 특화(specialization) 사이의 최적 트레이드오프 지점은 어디일까요?
A: 답변: Mixture-of-Experts(MoE) 모델의 성능 확장에 있어 효율적인 라우팅 전략은 핵심적인 연구 주제입니다. 현재 널리 사용되는 Top-k 게이팅은 단순성과 효율성에도 불구하고, 훈련 데이터에 특정 유형의 토큰(예: 코드, 전문 용어)이 많은 경우 해당 토큰을 처리하는 소수의 전문가에게 부하가 집중되는 '라우팅 편중' 현상을 야기합니다. 이는 모델의 일반화 성능 저하와 유휴 전문가 발생이라는 자원 비효율성 문제로 직결되므로, 토큰의 고유한 모달리티 특성을 반영한 동적 라우팅 알고리즘의 설계는 MoE 모델의 잠재력을 최대한 이끌어내기 위한 필수적인 과제라 할 수 있습니다.

이러한 라우팅 편중 현상을 완화하기 위해 몇 가지 고도화된 동적 라우팅 알고리즘을 설계할 수 있습니다. 첫째, '모달리티 인식 라우팅(Modality-Aware Routing)'을 도입하여, 기존의 토큰 임베딩 정보에 더해 해당 토큰이 코드, 자연어, 비전 데이터 중 무엇에 해당하는지 명시적인 모달리티 정보를 라우터의 입력으로 함께 사용하는 방식입니다. 둘째, 보조 손실 함수(Auxiliary Loss Function)를 정교화하여, 단순히 전체 전문가 간의 부하 분산을 유도하는 것을 넘어 각 모달리티 그룹 내에서 전문가 활용도를 개별적으로 측정하고 이를 기반으로 손실을 계산함으로써, 특정 모달리티 처리에 대한 과도한 전문화를 방지하고 유연성을 높일 수 있습니다. 셋째, '용량 인식 라우팅(Capacity-Aware Routing)'을 결합하여, 라우

In [83]:
for i, q in enumerate(questions):  # 전체 다 돌림
    h1, h2, question = q["Header 1"], q["Header 2"], q["Question"]

    # md_header_splits에서 헤더 매칭해서 본문 불러오기
    content = ""
    for doc in md_header_splits:
        if doc.metadata.get("Header 1") == h1 and doc.metadata.get("Header 2") == h2:
            content = doc.page_content
            break

    answer = generate_answer(question, h1, h2, content)
    q["Answer"] = answer  # 답변 추가

    print(f"\nQ{i+1}/{len(questions)}: {question}")
    print(f"A: {answer[:]}...")  # 앞부분만 미리보기 
    
    
    
import csv

output_path = "techreader_data/header_based_questions_with_answers.csv"

with open(output_path, "w", newline="", encoding="utf-8-sig") as f:
    fieldnames = ["Header 1", "Header 2", "Question", "Answer"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

    for q in questions:
        writer.writerow({
            "Header 1": q["Header 1"],
            "Header 2": q["Header 2"],
            "Question": q["Question"],
            "Answer": q.get("Answer", "")
        })

print(f"✅ 최종 저장 완료: {output_path}")




Q1/86: (라우팅 및 부하 분산)** 현재 MoE 모델에서 사용되는 Top-k 게이팅 기반 라우팅 전략이 특정 유형의 토큰(예: 코드, 비전 데이터)에 편중되는 현상을 어떻게 완화할 수 있을까요? 토큰의 모달리티(modality) 특성을 반영한 동적 라우팅(dynamic routing) 알고리즘을 설계한다면, 전문가 간 로드 밸런싱과 전문가 특화(specialization) 사이의 최적 트레이드오프 지점은 어디일까요?
A: 답변: 현재 MoE(Mixture-of-Experts) 모델에서 널리 사용되는 Top-k 게이팅 기반 라우팅 전략은 특정 유형의 토큰, 예를 들어 코드나 비전 데이터와 같은 특정 모달리티에 대한 전문가 편중 현상을 야기하여 모델의 효율성과 성능을 저해하는 주요 원인으로 지목됩니다. 이러한 편중은 특정 전문가에게 계산 부하를 집중시켜 라우팅 병목 현상을 유발하고, 다른 전문가들의 활용도를 저하시켜 전체 모델의 잠재력을 완전히 이끌어내지 못하게 만듭니다. 따라서, 다양한 데이터 모달리티를 효율적으로 처리하면서도 전문가 간의 부하를 균등하게 분배하는 고도화된 라우팅 메커니즘의 설계는 차세대 대규모 MoE 모델의 핵심적인 연구 과제라 할 수 있습니다.

이러한 문제를 완화하기 위해 토큰의 모달리티 특성을 명시적으로 반영하는 동적 라우팅 알고리즘을 설계할 수 있습니다. 기존의 Top-k 게이팅이 주로 토큰의 의미적 임베딩에만 의존했다면, 제안되는 알고리즘은 '모달리티 식별 메커니즘'을 라우팅 과정에 통합합니다. 예를 들어, 게이팅 네트워크의 입력으로 토큰 임베딩과 함께 해당 토큰이 텍스트, 코드, 이미지 중 어떤 유형에 속하는지를 나타내는 원핫 인코딩된 모달리티 벡터를 추가로 제공하는 방식입니다. 더 나아가, 훈련 과정에서 특정 모달리티의 토큰이 소수의 전문가에게 집중되는 것을 방지하기 위해 '모달리티 기반 부하 분산 손실(modality-aware load balancing loss)' 항을 전체 손실 함수에 추가할 수 있습니다. 이 손실 항

# 예상 질문 답변 쌍 저장 완료(csv 파일 2개)

In [ ]:
FAQ 리트리버 → 예상 질문-답변 바로 검색

원문 리트리버 → 새로운 질문 대응

하이브리드 → FAQ 매칭률 높으면 바로 답변, 아니면 원문 리트리버

In [ ]:
# faq 리트리버 성공 

In [85]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import pandas as pd

# 1. CSV 불러오기
df = pd.read_csv("techreader_data/content_based_questions_with_answers.csv")

# 2. 임베딩 생성
embeddings = OpenAIEmbeddings()  # 또는 HuggingFaceEmbeddings
docs = [q for q in df["Question"].tolist()]

# 3. FAISS 인덱스 만들기
db = FAISS.from_texts(docs, embeddings)

# 4. 리트리버 만들기
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# 5. 검색 함수
def retrieve_answer(user_query):
    results = retriever.get_relevant_documents(user_query)
    top_q = results[0].page_content  # 가장 가까운 질문
    answer = df[df["Question"] == top_q]["Answer"].values[0]
    return top_q, answer

# 예시 실행
q, a = retrieve_answer("실시간 RAG 파이프라인은 어떻게 설계하나?")
print("유사 질문:", q)
print("답변:", a)


C:\Users\SBA\AppData\Local\Temp\ipykernel_17400\2583537241.py:20: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(user_query)


유사 질문: 1. [RAG 아키텍처] 사내 데이터베이스와의 실시간 연동을 위한 최적의 RAG(Retrieval-Augmented Generation) 파이프라인 설계 방안은 무엇인가?
답변: 답변:
기업 환경에서 대규모 언어 모델(LLM)의 활용 가치를 극대화하기 위해 사내 데이터베이스와의 실시간 연동은 핵심적인 과제로 부상하고 있습니다. 정적인 문서 기반의 전통적인 RAG(Retrieval-Augmented Generation) 방식은 실시간으로 변동하는 재고, 고객 정보, 재무 데이터 등을 정확히 반영하지 못하는 명백한 한계를 가집니다. 따라서 LLM이 최신 데이터를 기반으로 신뢰도 높은 답변을 생성하게 하려면, 데이터의 동적 특성을 파이프라인 설계 단계부터 고려하는 고도화된 접근법이 필수적이며, 이는 기업의 의사결정 지원 및 업무 자동화 시스템 구축에 있어 매우 중요한 요소입니다.

본 보고서에서 분석한 최적의 RAG 파이프라인은 ‘쿼리 분석 기반 적응형 검색(Query-Aware Adaptive Retrieval)’ 아키텍처에 해당합니다. 이 모델은 사용자 질의의 의도를 먼저 파악하는 ‘쿼리 라우터(Query Router)’를 파이프라인 전면에 배치하는 것이 핵심입니다. 쿼리 라우터는 질의가 정적인 정보(예: 회사 정책 문서)를 요구하는지, 아니면 동적인 실시간 데이터(예: 현재 재고 수량)를 요구하는지를 판단합니다. 질의가 동적 데이터를 필요로 할 경우, 파이프라인은 벡터 검색 대신 Text-to-SQL과 같은 모델을 호출하여 사내 데이터베이스에 직접 SQL 쿼리를 실행하고 그 결과를 즉시 가져옵니다. 반면, 정적인 정보가 필요할 경우에는 기존 방식대로 벡터 데이터베이스에서 관련 문서를 검색하며, 두 가지 정보를 모두 요구하는 복합 질의에 대해서는 병렬적으로 두 경로를 모두 실행한 후 결과를 통합하여 LLM에 전달합니다. 이 방식은 불필요한 벡터 검색을 줄여 응답 속도를 개선하고, 데이터베이스에 직접 접근함으로써 정보의 최신성과 정확성을 극대화하는 효

In [ ]:
# 새로운 질문 리트리버  
# 하이브리드 (FAQ + 원문) (서비스 지향)

In [87]:
import os
import google.generativeai as genai
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
import pandas as pd

# 0. Gemini 설정
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
gemini_model = genai.GenerativeModel("gemini-2.5-pro")

# 1. CSV 불러오기 (FAQ 데이터셋)
df = pd.read_csv("techreader_data/content_based_questions_with_answers.csv")

# 2. FAQ 인덱스 생성
embeddings = OpenAIEmbeddings()
faq_questions = df["Question"].tolist()
faq_answers = df["Answer"].tolist()
faq_docs = [Document(page_content=q) for q in faq_questions]

faq_db = FAISS.from_documents(faq_docs, embeddings)
faq_retriever = faq_db.as_retriever(search_type="similarity", search_kwargs={"k": 1})

# 3. 원문 인덱스 (보고서 chunk md_header_splits 활용)
report_docs = []
for idx, doc in enumerate(md_header_splits):
    meta = doc.metadata
    report_docs.append(
        Document(
            page_content=doc.page_content,
            metadata={
                "Header 1": meta.get("Header 1", ""),
                "Header 2": meta.get("Header 2", ""),
                "Page": meta.get("page", idx+1)  # 페이지 번호 없으면 idx+1
            }
        )
    )

report_db = FAISS.from_documents(report_docs, embeddings)
report_retriever = report_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# 4. 하이브리드 검색 함수
def hybrid_answer(user_query, threshold=0.8):
    # (A) FAQ 검색
    faq_result = faq_db.similarity_search_with_score(user_query, k=1)[0]
    faq_doc, faq_score = faq_result[0], faq_result[1]

    if faq_score >= threshold:
        matched_q = faq_doc.page_content
        matched_a = df[df["Question"] == matched_q]["Answer"].values[0]
        return f"[FAQ 기반 응답]\nQ: {matched_q}\nA: {matched_a}"

    else:
        # (B) 원문 검색 → Gemini 답변
        report_results = report_retriever.get_relevant_documents(user_query)
        context = "\n\n".join([r.page_content for r in report_results])

        # 출처 정보
        sources = []
        for r in report_results:
            h1 = r.metadata.get("Header 1", "")
            h2 = r.metadata.get("Header 2", "")
            page = r.metadata.get("Page", "")
            sources.append(f"- {h1} > {h2} (p.{page})")

        prompt = f"""
        다음 질문에 대해 아래 보고서 본문을 근거로 연구 보고서 스타일로 답변하라.
        반드시 근거 내용을 참고하되, 마지막에 한 문장으로 요약 결론을 제시하라.

        질문: {user_query}

        본문 발췌:
        {context}
        """

        response = gemini_model.generate_content(
            prompt,
            generation_config={"max_output_tokens": 2048, "temperature": 0.7}
        )

        if response.candidates and response.candidates[0].content.parts:
            answer = response.candidates[0].content.parts[0].text.strip()
        else:
            answer = "[⚠️ 답변 없음: Gemini 출력 실패]"

        return f"[원문 기반 응답]\n{answer}\n\n📌 출처:\n" + "\n".join(sources)


In [ ]:
# 실행 스크립트 (FAQ CSV + 보고서 chunk → 하이브리드 QA)
아래 코드를 하나의 Python 파일 (hybrid_qa.py)로 저장해 실행하면,
FAQ CSV와 md_header_splits(보고서 chunk)를 동시에 불러와서 바로 질문–응답 테스트까지 돌아갑니다.

위 스크립트를 저장 (hybrid_qa.py)
md_header_splits 변수가 로드된 환경에서 실행
(예: LlamaParse + MarkdownHeaderTextSplitter로 미리 생성)

FAQ 기반 매칭이 높으면 CSV의 답변 출력
매칭이 낮으면 보고서 chunk에서 근거를 검색해 Gemini가 답변 생성

In [ ]:
import os
import pandas as pd
import google.generativeai as genai
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document

# ==============================
# 0. Gemini API 초기화
# ==============================
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
gemini_model = genai.GenerativeModel("gemini-2.5-pro")

# ==============================
# 1. FAQ CSV 불러오기
# ==============================
df = pd.read_csv("techreader_data/content_based_questions_with_answers.csv")

# ==============================
# 2. FAQ 인덱스 만들기
# ==============================
embeddings = OpenAIEmbeddings()
faq_questions = df["Question"].tolist()
faq_docs = [Document(page_content=q) for q in faq_questions]

faq_db = FAISS.from_documents(faq_docs, embeddings)

# ==============================
# 3. 원문 인덱스 만들기 (md_header_splits 필요)
# ==============================
# ⚠️ md_header_splits는 미리 준비된 chunk 리스트여야 합니다.
#    각 요소는 page_content와 metadata(Header 1, Header 2, page 등)를 포함해야 합니다.

report_docs = []
for idx, doc in enumerate(md_header_splits):  # md_header_splits는 사전에 로드되어 있어야 함
    meta = doc.metadata
    report_docs.append(
        Document(
            page_content=doc.page_content,
            metadata={
                "Header 1": meta.get("Header 1", ""),
                "Header 2": meta.get("Header 2", ""),
                "Page": meta.get("page", idx+1)
            }
        )
    )

report_db = FAISS.from_documents(report_docs, embeddings)

# ==============================
# 4. 하이브리드 검색 함수
# ==============================
def hybrid_answer(user_query, threshold=0.8):
    # (A) FAQ 검색
    faq_result = faq_db.similarity_search_with_score(user_query, k=1)[0]
    faq_doc, faq_score = faq_result[0], faq_result[1]

    if faq_score >= threshold:
        matched_q = faq_doc.page_content
        matched_a = df[df["Question"] == matched_q]["Answer"].values[0]
        return f"[FAQ 기반 응답]\nQ: {matched_q}\nA: {matched_a}"

    else:
        # (B) 원문 검색
        report_results = report_db.similarity_search(user_query, k=3)
        context = "\n\n".join([r.page_content for r in report_results])

        sources = []
        for r in report_results:
            h1 = r.metadata.get("Header 1", "")
            h2 = r.metadata.get("Header 2", "")
            page = r.metadata.get("Page", "")
            sources.append(f"- {h1} > {h2} (p.{page})")

        prompt = f"""
        다음 질문에 대해 아래 보고서 본문을 근거로 연구 보고서 스타일로 답변하라.
        반드시 근거 내용을 참고하되, 마지막에 결론 문장을 추가하라.

        질문: {user_query}

        본문 발췌:
        {context}
        """

        response = gemini_model.generate_content(
            prompt,
            generation_config={"max_output_tokens": 9048, "temperature": 0.7}
        )

        if response.candidates and response.candidates[0].content.parts:
            answer = response.candidates[0].content.parts[0].text.strip()
        else:
            answer = "[⚠️ 답변 없음: Gemini 출력 실패]"

        return f"[원문 기반 응답]\n{answer}\n\n📌 출처:\n" + "\n".join(sources)

# ============================== 
# 5. 실행 예시
# ==============================
if __name__ == "__main__":
    user_query = "실시간 RAG 파이프라인은 어떻게 설계하나?"
    print(hybrid_answer(user_query))


[원문 기반 응답]
## 실시간 RAG 파이프라인 설계 방안 연구

본 보고서는 제시된 본문을 근거로 실시간 RAG(Retrieval-Augmented Generation) 파이프라인의 핵심 설계 단계를 기술하고, 성능 향상을 위한 확장 아키텍처를 제시한다.

### 1. RAG 파이프라인의 기본 설계
RAG 파이프라인은 크게 정보의 벡터화 및 저장, 관련 정보 검색, 컨텍스트 보강 및 답변 생성의 3단계로 설계된다. 이는 사용자가 외부 문서를 검색

📌 출처:
- LLM 한계 극복을 위한 RAG의 역할과 최신 동향 > RAG 개선하기 (p.15)
- LLM 한계 극복을 위한 RAG의 역할과 최신 동향 > 해결책 : LLM과 사실의 그라운딩 (p.14)
- AI 코딩, LLM 혼합 전략이 답이다 >  (p.28)


In [ ]:
# 답변이 짤리는 현상 발생 
이럴 바에는 미리 만들어둔 예상질문 답변쌍의 답변을 가져오고 출처만 붙여주는 것이 낫다.  

그래서 말씀처럼 예상질문 답변쌍의 답변을 가져오고, 거기에 출처만 붙여주는 방식이 더 안정적이고 효율적입니다.


In [89]:
def hybrid_answer(user_query, threshold=0.8):
    # (A) FAQ 검색
    faq_result = faq_db.similarity_search_with_score(user_query, k=1)[0]
    faq_doc, faq_score = faq_result[0], faq_result[1]

    if faq_score >= threshold:
        matched_q = faq_doc.page_content
        matched_a = df[df["Question"] == matched_q]["Answer"].values[0]

        # 해당 질문과 매칭되는 chunk 찾아 출처 달기
        sources = []
        for doc in md_header_splits:
            if matched_q in doc.page_content:  # 단순 매칭 (필요시 cosine 기반 보강)
                h1 = doc.metadata.get("Header 1", "")
                h2 = doc.metadata.get("Header 2", "")
                page = doc.metadata.get("page", "?")
                sources.append(f"- {h1} > {h2} (p.{page})")
        sources_text = "\n".join(sources) if sources else "출처를 찾을 수 없음"

        return f"[FAQ 기반 응답]\nQ: {matched_q}\nA: {matched_a}\n\n📌 출처:\n{sources_text}"

    else:
        # (B) 원문 검색 + Gemini 호출
        report_results = report_db.similarity_search(user_query, k=3)
        context = "\n\n".join([r.page_content for r in report_results])

        sources = []
        for r in report_results:
            h1 = r.metadata.get("Header 1", "")
            h2 = r.metadata.get("Header 2", "")
            page = r.metadata.get("Page", "")
            sources.append(f"- {h1} > {h2} (p.{page})")

        prompt = f"""
        다음 질문에 대해 아래 보고서 본문을 근거로 연구 보고서 스타일로 답변하라.
        반드시 근거 내용을 참고하되, 마지막에 결론 문장을 추가하라.

        질문: {user_query}

        본문 발췌:
        {context}
        """

        response = gemini_model.generate_content(
            prompt,
            generation_config={"max_output_tokens": 2048, "temperature": 0.7}
        )

        if response.candidates and response.candidates[0].content.parts:
            answer = response.candidates[0].content.parts[0].text.strip()
        else:
            answer = "[⚠️ 답변 없음: Gemini 출력 실패]"

        return f"[원문 기반 응답]\n{answer}\n\n📌 출처:\n" + "\n".join(sources)
    
    # ============================== 
# 5. 실행 예시 
# ==============================
if __name__ == "__main__":
    user_query = "실시간 RAG 파이프라인은 어떻게 설계하나?"
    print(hybrid_answer(user_query))



[원문 기반 응답]
## 실시간 RAG 파이프라인 설계 방안 연구

보고서에 따르면, 실시간 검색 증강 생성(R

📌 출처:
- LLM 한계 극복을 위한 RAG의 역할과 최신 동향 > RAG 개선하기 (p.15)
- LLM 한계 극복을 위한 RAG의 역할과 최신 동향 > 해결책 : LLM과 사실의 그라운딩 (p.14)
- AI 코딩, LLM 혼합 전략이 답이다 >  (p.28)


In [ ]:
# 그래도 여전히 답변이 불완전하다. 
이럴거면 faq는 아예 사용자 화면에 보여주고, 그 안에서 세부 사용자 검색이 들어가도록 하는 게 낫지 

🔹 개선된 UX 흐름

FAQ 목록을 사용자 화면에 그대로 제공

주제/소제목 기준으로 FAQ 그룹핑

각 항목마다 예상 질문 5~10개 표시

사용자 검색

사용자가 자유롭게 질문 → FAQ 질문과 임베딩 검색

가장 가까운 질문–답변 페어 반환

출처 제공

FAQ에 이미 연결된 Header1/2, Page 메타데이터를 같이 노출

In [98]:
import gradio as gr
import pandas as pd
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document

# 1. CSV 로드
df = pd.read_csv("techreader_data/content_based_questions_with_answers.csv")

# 2. FAQ 인덱스 생성
embeddings = OpenAIEmbeddings()
faq_docs = [
    Document(page_content=row["Question"], metadata=row.to_dict())
    for _, row in df.iterrows()
]
faq_db = FAISS.from_documents(faq_docs, embeddings)
faq_retriever = faq_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# 3. 검색 함수
def search_faq(user_query):
    results = faq_retriever.get_relevant_documents(user_query)
    outputs = []
    for r in results:
        q = r.page_content
        a = r.metadata["Answer"]
        h1 = r.metadata.get("Header 1", "")
        h2 = r.metadata.get("Header 2", "")
        outputs.append(f"📌 **Q:** {q}\n\n**A:** {a}\n\n출처: {h1} > {h2}\n")
    return "\n---\n".join(outputs)

# 2. FAQ 뷰어 함수 (Accordion 형식)
def show_faq():
    grouped = df.groupby("Header 1")
    for h1, group in grouped:
        with gr.Accordion(f"📘 {h1}", open=False):
            sub_group = group.groupby("Header 2")
            for h2, rows in sub_group:
                if h2 and h2 != "nan":
                    with gr.Accordion(f"📌 {h2}", open=False):
                        for _, row in rows.iterrows():
                            # 질문을 Accordion 제목으로, 답변은 내부 내용으로
                            with gr.Accordion(f"💡 {row['Question']}", open=False):
                                gr.Markdown(f"📖 {row['Answer']}")
                else:
                    for _, row in rows.iterrows():
                        with gr.Accordion(f"💡 {row['Question']}", open=False):
                            gr.Markdown(f"📖{row['Answer']}")

with gr.Blocks() as demo:
    gr.Markdown("## 📘 Tech Library FAQ 뷰어")
    with gr.Tab("FAQ 검색"):
        query = gr.Textbox(label="질문을 입력하세요")
        output = gr.Markdown()
        # 검색 기능은 기존 search_faq 함수 연결
        query.submit(search_faq, query, output)

    with gr.Tab("FAQ 전체 보기"):
        show_faq()

demo.launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


In [110]:
import gradio as gr
import pandas as pd
import re
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document

# 1. CSV 로드
df = pd.read_csv("techreader_data/content_based_questions_with_answers.csv")

# 2. FAQ 인덱스 생성
embeddings = OpenAIEmbeddings()
faq_docs = [
    Document(page_content=row["Question"], metadata=row.to_dict())
    for _, row in df.iterrows()
]
faq_db = FAISS.from_documents(faq_docs, embeddings)
faq_retriever = faq_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# 3. '답변:' 접두사 제거 함수
def clean_answer(text: str) -> str:
    if text.strip().startswith("답변"):
        return text.split(":", 1)[-1].strip()
    return text.strip()

# 4. 질문 후처리 함수 (**텍스트** 제거, > 제거)
def clean_question(q: str) -> str:
    # 1) **텍스트** → 텍스트
    q = re.sub(r"\*\*(.*?)\*\*", r"\1", q)
    # 2) ** 텍스트 → 텍스트  (앞에 ** + 공백 제거)
    q = re.sub(r"\*\*\s*", "", q)
    # 3) > 텍스트 → 텍스트
    q = re.sub(r"^\s*>\s*", "", q)
    return q.strip()



# 5. 카드 스타일 FAQ 템플릿
def format_faq_card(q, a, h1="", h2=""):
    return f"""
    <div style="margin:15px 10px; border:1px solid #ddd; border-radius:10px; overflow:hidden;">
      <!-- 질문 영역 -->
      <details>
        <summary style="padding:12px; background:#f5f5f5; cursor:pointer; display:flex; align-items:center;">
          <div style="background:#1976d2; color:white; font-weight:bold;
                      border-radius:50%; width:32px; height:32px;
                      display:flex; align-items:center; justify-content:center;
                      margin-right:10px; font-size:16px; line-height:32px;">Q</div>
          <span style="font-weight:bold; font-size:16px;">{q}</span>
        </summary>
        
        <!-- 답변 영역 -->
        <div style="padding:15px; background:white; font-size:15px; line-height:1.6;">
          {a}
        </div>
      </details>
    </div>
    """

# 6. FAQ 검색
def search_faq(user_query):
    results = faq_retriever.get_relevant_documents(user_query)
    outputs = []
    for r in results:
        q = clean_question(r.page_content)         # ✅ 질문 후처리 적용
        a = clean_answer(r.metadata["Answer"])     # ✅ 답변 후처리 적용
        h1 = r.metadata.get("Header 1", "")
        h2 = r.metadata.get("Header 2", "")
        outputs.append(format_faq_card(q, a, h1, h2))
    return "".join(outputs)

# 7. FAQ 전체 보기
def show_faq():
    grouped = df.groupby("Header 1")
    html_blocks = []
    for h1, group in grouped:
        html_blocks.append(f"<h2 style='color:#1976d2; margin-top:40px;'>📘 {h1}</h2>")
        sub_group = group.groupby("Header 2")
        for h2, rows in sub_group:
            if h2 and h2 != "nan":
                html_blocks.append(f"<h3 style='color:#444; margin-top:20px;'>📌 {h2}</h3>")
            for _, row in rows.iterrows():
                q = clean_question(row["Question"])  # ✅ 질문 후처리 적용
                a = clean_answer(row["Answer"])      # ✅ 답변 후처리 적용
                html_blocks.append(format_faq_card(q, a, h1, h2))
    return "".join(html_blocks)

# 8. Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 📘 Tech Library FAQ 뷰어")

    with gr.Tab("FAQ 검색"):
        query = gr.Textbox(label="질문을 입력하세요")
        output = gr.HTML()
        query.submit(search_faq, query, output)

    with gr.Tab("FAQ 전체 보기"):
        faq_output = gr.HTML(show_faq())

demo.launch()


* Running on local URL:  http://127.0.0.1:7880
* To create a public link, set `share=True` in `launch()`.


![FAQ](img/0904_faq.png)


In [ ]:
# 그러면 예상 질문 쌍을 만들었으면, 답변은 그대로 사용하고, 해당 예상 질문들과 paraphrase하는 구문들도 만들어서 넣는게 낫나???
지금은 예상 질문 → 답변 1:1 매칭만 되어 있는데, 실제 서비스/FAQ 검색 품질을 생각하면 질문 다양화(paraphrase) 가 엄청 중요합니다.

사용자는 FAQ 질문을 똑같이 입력하지 않음 → 표현이 다르면 검색 매칭률이 떨어짐
같은 의미를 가진 다양한 구문을 미리 추가해두면 검색기 recall이 올라감
 
 
궁금한 사항을 매번 검색하면서, 아주 정확하지 않더라도 비슷하면 보고서를 읽는 것에 대한 만족감 향상 


In [114]:
import pandas as pd
import os
import google.generativeai as genai

# Gemini 초기화
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-2.5-pro")

def generate_paraphrases(question, n=3):
    prompt = f"""
    다음 질문을 의미가 동일하지만 표현이 다른 방식으로 {n}개 만들어줘.
    질문: {question}
    """
    try:
        response = model.generate_content(prompt)
        if response.candidates and response.candidates[0].content.parts:
            text = response.candidates[0].content.parts[0].text.strip()
            # 줄바꿈/불릿 제거
            paras = [line.strip(" -•0123456789.") for line in text.split("\n") if line.strip()]
            return paras
    except Exception as e:
        print(f"⚠️ 오류 발생: {e}")
    return []

# 원본 CSV 로드
input_path = "techreader_data/content_based_questions_with_answers.csv"
df = pd.read_csv(input_path)

# Paraphrases 컬럼 추가 (기존 데이터는 훼손하지 않음)
df["Paraphrases"] = df["Question"].apply(lambda q: generate_paraphrases(q, n=3))

# 새 파일로 저장
output_path = "techreader_data/content_based_FAQ_with_paraphrases.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"✅ Paraphrases 추가 완료: {output_path}")


✅ Paraphrases 추가 완료: techreader_data/content_based_FAQ_with_paraphrases.csv


In [116]:
# 위의 paraphrase된 질문에는 질문을 포함한 광범위한 내용이 담겨 필터링 필요 - 우선 시범 테스트 진행 
import pandas as pd
import os
import google.generativeai as genai

# Gemini 초기화
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-2.5-pro")

def generate_paraphrases(question, n=5):
    prompt = f"""
    다음 질문을 의미가 동일하지만 표현이 다른 방식으로 {n}개 만들어줘.
    출력은 반드시 질문만, 각 줄 하나씩, 불필요한 설명이나 번호, 불릿, 마크다운 기호 없이 작성해.
    질문: {question}
    """
    try:
        response = model.generate_content(prompt)
        if response.candidates and response.candidates[0].content.parts:
            text = response.candidates[0].content.parts[0].text.strip()
            # 줄 단위 split
            lines = [line.strip(" -•0123456789.") for line in text.split("\n") if line.strip()]
            # "?" 로 끝나는 질문만 남김
            paras = [line for line in lines if line.endswith("?")]
            return paras
    except Exception as e:
        print(f"⚠️ 오류 발생: {e}")
    return []

# 원본 CSV 로드
input_path = "techreader_data/content_based_questions_with_answers.csv"
df = pd.read_csv(input_path)

# 상위 3개 질문만 테스트
sample_questions = df["Question"].head(3).tolist()

print("🔹 Paraphrase 생성 테스트 (3개 질문)\n")
for i, q in enumerate(sample_questions, start=1):
    paras = generate_paraphrases(q, n=3)
    print(f"Q{i}: {q}")
    for j, p in enumerate(paras, start=1):
        print(f"   → P{j}: {p}") 
    print("-" * 60)


🔹 Paraphrase 생성 테스트 (3개 질문)

Q1: 1. [RAG 아키텍처] 사내 데이터베이스와의 실시간 연동을 위한 최적의 RAG(Retrieval-Augmented Generation) 파이프라인 설계 방안은 무엇인가?
   → P1: 사내 데이터베이스의 최신 정보를 실시간으로 반영하는 RAG 파이프라인을 구축하는 가장 효과적인 전략은 무엇입니까?
------------------------------------------------------------
Q2: 실시간으로 변경되는 벡터 데이터베이스(Vector DB)의 인덱싱 지연을 최소화하고, LLM(거대 언어 모델)이 항상 최신 정보를 참조하도록 보장하려면 어떤 기술 스택(e.g., CDC, Incremental Indexing) 조합이 가장 효과적일까요? 정확성과 응답 속도 간의 트레이드오프는 어떻게 관리해야 할까요?
   → P1: LLM이 최신 정보를 기반으로 응답하도록, 동적으로 변하는 벡터 DB의 인덱싱 지연을 줄이기 위한 최적의 기술 조합(CDC, 증분 인덱싱 등)은 무엇이며, 정보 최신성과 검색 속도 간의 균형점은 어떻게 찾아야 할까요?
   → P2: 지속적으로 업데이트되는 벡터 저장소와 LLM 간의 데이터 동기화 지연을 최소화하기 위한 가장 효율적인 아키텍처는 무엇이며, 이 과정에서 발생하는 데이터 신선도와 쿼리 성능의 상충 관계는 어떻게 해결해야 하나요?
   → P3: 실시간 데이터 변경을 벡터 DB에 즉각 반영하여 LLM이 항상 최신 상태를 참조하게 만드는 기술 스택은 어떻게 구성해야 하며, 응답 속도를 희생하지 않으면서 데이터 정확성을 최대로 확보하는 전략은 무엇인가요?
------------------------------------------------------------
Q3: 2. [LLM 서빙 최적화] 자체 호스팅(On-premise/VPC) LLM의 추론(Inference) 비용을 현재의 50% 수준으로 절감하기 위한 구체적인 최적화 전략은 무엇인가?
   →

In [118]:
import pandas as pd
import os
import google.generativeai as genai

# Gemini 초기화
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-2.5-pro")

def generate_paraphrases(question, n=4):
    prompt = f"""
    다음 질문을 의미가 동일하지만 표현이 다른 방식으로 {n}개 만들어줘.
    출력은 반드시 질문만, 각 줄 하나씩, 불필요한 설명이나 번호, 불릿, 마크다운 기호 없이 작성해.
    질문: {question}
    """
    try:
        response = model.generate_content(prompt)
        if response.candidates and response.candidates[0].content.parts:
            text = response.candidates[0].content.parts[0].text.strip()
            lines = [line.strip(" -•0123456789.") for line in text.split("\n") if line.strip()]
            paras = [line for line in lines if line.endswith("?")]  # 질문만
            return paras
    except Exception as e:
        print(f"⚠️ 오류 발생: {e}")
    return []

# 원본 CSV 로드
input_path = "techreader_data/content_based_questions_with_answers.csv"
df = pd.read_csv(input_path)

# Paraphrases 생성
df["Paraphrases"] = df["Question"].apply(lambda q: generate_paraphrases(q, n=4))

# 새 파일로 저장
output_path = "techreader_data/content_based_FAQ2_with_paraphrases.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"✅ Paraphrases 추가 완료: {output_path}")


✅ Paraphrases 추가 완료: techreader_data/content_based_FAQ2_with_paraphrases.csv


In [119]:
# TechReader_gayoon/techreader_data/header_based_questions_with_answers.csv 

import pandas as pd
import os
import google.generativeai as genai

# Gemini 초기화
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-2.5-pro")

def generate_paraphrases(question, n=4):
    prompt = f"""
    다음 질문을 의미가 동일하지만 표현이 다른 방식으로 {n}개 만들어줘.
    출력은 반드시 질문만, 각 줄 하나씩, 불필요한 설명이나 번호, 불릿, 마크다운 기호 없이 작성해.
    질문: {question}
    """
    try:
        response = model.generate_content(prompt)
        if response.candidates and response.candidates[0].content.parts:
            text = response.candidates[0].content.parts[0].text.strip()
            lines = [line.strip(" -•0123456789.") for line in text.split("\n") if line.strip()]
            paras = [line for line in lines if line.endswith("?")]  # 질문만
            return paras
    except Exception as e:
        print(f"⚠️ 오류 발생: {e}")
    return []

# 원본 CSV 로드
input_path = "techreader_data/header_based_questions_with_answers.csv "
df = pd.read_csv(input_path)

# Paraphrases 생성
df["Paraphrases"] = df["Question"].apply(lambda q: generate_paraphrases(q, n=4))

# 새 파일로 저장
output_path = "techreader_data/header_based_FAQ2_with_paraphrases.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"✅ Paraphrases 추가 완료: {output_path}")


✅ Paraphrases 추가 완료: techreader_data/header_based_FAQ2_with_paraphrases.csv


In [ ]:
# 문장 다양화 작업 모두 끝나면 다음 단계를 위해 아래 코드 테스트해본 후 성공적이면 아래 문장 지시 (chatgpt)
이렇게 하면 이제 FAQ + Paraphrases → 임베딩 → 벡터스토어 → Retriever 구조까지 완성됩니다.
여기서 다음 단계는, 이 retriever를 Gradio UI FAQ 검색기에 연결해서 사용자 입력 → 유사 질문 매칭 → 답변 반환 플로우를 구성하는 겁니다.

원하시면 제가 현재 Gradio FAQ 뷰어 코드랑 지금 만든 retriever를 합쳐드릴까요?

In [ ]:
faq 질문 답변쌍은 CacheBackedEmbeddings 사용 
일반 질문 답변쌍은 업스테이지 임베딩 모델 사용 


![임베딩모델선택](img/임베딩모델비교.png)

In [ ]:
# 👉 이렇게 하면 
FAQ + Paraphrases는 CacheBackedEmbeddings로 캐싱,
원문 Chunk는 UpstageEmbeddings로 임베딩,
두 인덱스를 동시에 운용할 수 있습니다.

In [121]:
!pip install langchain-upstage

  Using cached tokenizers-0.20.3-cp311-none-win_amd64.whl.metadata (6.9 kB)
Using cached tokenizers-0.20.3-cp311-none-win_amd64.whl (2.4 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-chroma 0.2.2 requires chromadb!=0.5.10,!=0.5.11,!=0.5.12,!=0.5.4,!=0.5.5,!=0.5.7,!=0.5.9,<0.7.0,>=0.4.0, but you have chromadb 1.0.20 which is incompatible.
transformers 4.50.3 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.


In [123]:
import pandas as pd
import ast
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain_upstage import UpstageEmbeddings
from langchain.storage import LocalFileStore
from pathlib import Path

embeddings = UpstageEmbeddings(model="solar-embedding-1-large", api_key="UPSTAGE_API_KEY")


# ----------------------------
# 1. FAQ 데이터 (CacheBackedEmbeddings)
# ----------------------------
def load_faq_docs(file_path):
    df = pd.read_csv(file_path)
    docs = []
    for _, row in df.iterrows():
        base_q = row["Question"]
        paras = []
        try:
            paras = ast.literal_eval(row["Paraphrases"])
        except:
            pass
        all_qs = [base_q] + paras
        for q in all_qs:
            docs.append(Document(
                page_content=q,
                metadata={"Answer": row["Answer"],
                          "Header 1": row.get("Header 1", ""),
                          "Header 2": row.get("Header 2", "")}
            ))
    return docs

faq_files = [
    "techreader_data/header_based_FAQ2_with_paraphrases.csv",
    "techreader_data/content_based_FAQ2_with_paraphrases.csv"
]

faq_docs = []
for f in faq_files:
    faq_docs.extend(load_faq_docs(f))

# 캐시 디렉토리
cache_dir = Path("techreader_data/faq_cache")
store = LocalFileStore(str(cache_dir))

# 기본 임베딩
base_embeddings = OpenAIEmbeddings()

# CacheBackedEmbeddings 구성
faq_embeddings = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=base_embeddings,
    document_embedding_cache=store
)

faq_db = FAISS.from_documents(faq_docs, faq_embeddings)
faq_db.save_local("techreader_data/faq_index")
faq_retriever = faq_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

print("✅ FAQ Retriever 준비 완료")


✅ FAQ Retriever 준비 완료


In [ ]:
# chunks_output.csv -> chunk_docs 변환 결과 
page_content : 본문 텍스트 (chunk)
metadata : Header 1, Header 2/Header 3 같은 계층적 제목 + Chunk No

In [127]:
import pandas as pd
import ast
from langchain.schema import Document

chunks_df = pd.read_csv("techreader_data/chunks_output.csv")

chunk_docs = []
for _, row in chunks_df.iterrows():
    content = row["Content"]

    # Metadata 문자열 → dict 변환
    metadata = {}
    if isinstance(row["Metadata"], str) and row["Metadata"].strip() != "{}":
        try:
            metadata = ast.literal_eval(row["Metadata"])
        except Exception:
            metadata = {"raw_metadata": row["Metadata"]}

    # Chunk 번호도 추가
    metadata["Chunk No"] = row["Chunk No"]

    chunk_docs.append(Document(page_content=content, metadata=metadata))

print(f"✅ 총 {len(chunk_docs)}개 chunk 변환 완료")
print("예시 Document:", chunk_docs[0])


✅ 총 31개 chunk 변환 완료
예시 Document: page_content='IT WORLD CIO' metadata={'Chunk No': 1}


In [129]:
chunk_docs[25]

Document(metadata={'Header 1': 'AI 코딩, LLM 혼합 전략이 답이다', 'Header 3': '오픈AI o3 : 고급 문제 해결사, 가격도 고급', 'Chunk No': 26}, page_content='오픈AI의 o3(이름 때문에 GPT 시리즈로 오해하기 쉽다)은 연구용 추론 엔진이다. 도구 호출을 연쇄 실행하고 분석 보고서를 작성하며, 300개 테스트가 있는 제스트(Jest) 스위트를 아무 불평 없이 검토한다. 단, 접근 제한이 있고(나는 여권을 제출해야 했다), 느리며, 비싸다. 만약 사용자가 FAANG급 예산을 갖고 있거나, 스스로 버그를 해결할 수 없는 상황이 아니라면, o3은 일상용이 아닌 사치품이다.')

In [130]:
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings  # pip install langchain-upstage

# Upstage 임베딩 초기화
chunk_embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large", 
    api_key=os.environ["UPSTAGE_API_KEY"]
)

# FAISS 인덱스 구축
chunk_db = FAISS.from_documents(chunk_docs, chunk_embeddings)

# 로컬 저장
chunk_db.save_local("techreader_data/chunk_index")

# Retriever
chunk_retriever = chunk_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

print("✅ Chunk Retriever 준비 완료")


✅ Chunk Retriever 준비 완료


In [132]:
query = "교사와 학생 모델의 차이점은 무엇인가?"
results = chunk_retriever.get_relevant_documents(query)

for r in results:
    print("본문:", r.page_content[:200], "...")
    print("출처:", r.metadata)
    print("-" * 60)


본문: 학생 모델은 스스로 맥락을 이해하지 않고 교사 모델의 사전 학습된 결론에 크게 의존한다. 이런 제한이 모델 환각으로 이어질지에 대해서는 전문가 사이에서 많은 논란이 있다.  
브라우클러는 훈련 방식에 관계없이 학생 모델의 효율성은 곧 교사 모델의 효율성과 관련이 있다고 생각한다. 즉, 교사 모델에서 환각이 없으면 학생 모델에도 없을 가능성이 높다는 의미다. ...
출처: {'Header 1': 'LLM을 학습한 추출 모델, 작아도 위험은 동일', 'Header 2': '교사 모델 부담을 떠맡은 학생 모델', 'Header 3': '지혜를 전수받지만 취약성은 커져', 'Chunk No': 8}
------------------------------------------------------------
본문: 추출된 모델은 훈련 데이터에 내재된 보안 위험을 포함해 원래 모델의 행동 상당 부분을 그대로 물려받는다. 지적 재산권 도용, 개인 정보 유출, 모델 반전 공격 등의 위험을 그대로 떠안는 것이다.  
브라우클러는 “일반적으로 모델 추출은 원래 더 큰 교사 모델이 소비한 훈련 데이터와 교사 모델의 유효한 결과(결과의 확률 분포 등) 예측을 사용한다. 결과적으로 ...
출처: {'Header 1': 'LLM을 학습한 추출 모델, 작아도 위험은 동일', 'Header 2': '교사 모델 부담을 떠맡은 학생 모델', 'Chunk No': 7}
------------------------------------------------------------
본문: **Shweta Sharma | CSOonline**  
대형 언어 모델이 주류가 되면서 AI 기반 애플리케이션의 범위가 한층 더 확장되고, 그만큼 복잡성도 늘었다. 대가도 따른다. 현실적으로 대형 언어 모델은 비용은 높고 지연은 길어 실용성이 낮다.  
모델 추출 입력은 AI 엔지니어가 매개변수가 높은 모델의 가장 유용한 측면을 훨씬 작은 모방 모델에 담 ...
출처: {'Header 1': 'LLM을 학

# 2개의 리트리버 준비 완료 - FAQ 리트리버, Chunk 리트리버 

In [ ]:
그러면 사용자 구분을 해서 하이브리드 전략을 해야겠네 , 전문가는 faq에 해당되는 난이도 높은 질문을 할 것이고, 입문자나 일반 사용자는 chunk에 해당되는 질문을 할 것이고 
하지만 현재는 전문 사용자로 한정해서 진행할 것 



In [ ]:
1. Hybrid Retriever 설계

1차 검색: FAQ retriever

paraphrase 포함된 질문-답변쌍에서 hit가 나오면 그대로 FAQ 답변 반환

Fallback: Chunk retriever

FAQ에서 적절한 매칭이 없거나, 유사도가 낮을 경우 → 원문 기반 chunk 검색

→ 이렇게 하면 FAQ 우선 → 문서 보강 플로우가 됩니다. 

2. Scoring (FAQ vs Chunk)

두 retriever가 다 결과를 주면,

FAQ match score (cosine similarity)

Chunk match score 비교해서

FAQ가 threshold 이상이면 FAQ 답변, 아니면 chunk 기반 답변

FAQ_THRESHOLD = 0.75

3. Chunk 결과 → 답변 생성

FAQ는 이미 답변이 붙어 있음 ✅

Chunk는 retrieved content를 LLM에 넣어 답변 생성해야 함

프롬프트 예시:

prompt = f"""
아래 문서 조각들을 참고하여 질문에 답하세요. 
문서 내용만 활용하여 구체적이고 정확하게 답하세요.

질문: {query}
문서:
{retrieved_chunks}
"""

4. Gradio 통합 검색 UI

검색창 1개 (→ 내부에서 hybrid search 실행)

결과는 두 가지 형태:

📌 FAQ 매칭 → 질문/답변 바로 반환

📑 Chunk 기반 → 생성된 답변 + 출처(헤더/페이지) 같이 반환

In [133]:
import os
import google.generativeai as genai

# Gemini 초기화
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-2.5-pro")

FAQ_THRESHOLD = 0.75  # FAQ 신뢰도 기준값

# -----------------------------
# 1. FAQ 기반 응답
# -----------------------------
def format_faq_answer(doc):
    q = doc.page_content
    a = doc.metadata.get("Answer", "")
    h1 = doc.metadata.get("Header 1", "")
    h2 = doc.metadata.get("Header 2", "")
    return f"""📌 **FAQ 응답**
**Q:** {q}

**A:** {a}

출처: {h1} > {h2}
"""

# -----------------------------
# 2. Chunk 기반 응답 (LLM 다듬기)
# -----------------------------
def chunk_answer(query, retrieved_docs):
    # 관련 chunk 모으기
    docs_text = "\n\n".join(
        [f"[Chunk {doc.metadata.get('Chunk No')}] {doc.page_content}" for doc in retrieved_docs]
    )

    prompt = f"""
    너는 연구 보고서를 요약하는 LLM 어시스턴트다. 
    아래 문서 조각들만 참고해서 질문에 답변을 재구성하라. 
    문서에 없는 내용은 절대 만들지 말고, 출처(Chunk No 및 Header)를 반드시 포함하라.

    질문: {query}

    문서 조각:
    {docs_text}
    """

    try:
        response = model.generate_content(
            prompt,
            generation_config={"max_output_tokens": 800, "temperature": 0.5}
        )
        return "📑 **문서 기반 응답**\n" + response.text.strip()
    except Exception as e:
        return f"[⚠️ 오류 발생: {e}]"

# -----------------------------
# 3. Hybrid Search
# -----------------------------
def hybrid_search(query, faq_retriever, chunk_retriever):
    # 1. FAQ 검색
    faq_results = faq_retriever.get_relevant_documents(query)

    if faq_results:
        # retriever가 score 속성을 포함하는 경우
        score = getattr(faq_results[0], "score", 1.0)  
        if score >= FAQ_THRESHOLD:
            return format_faq_answer(faq_results[0])

    # 2. Chunk 검색 (Fallback)
    chunk_results = chunk_retriever.get_relevant_documents(query)
    if chunk_results:
        return chunk_answer(query, chunk_results)

    return "⚠️ 관련된 답변을 찾을 수 없습니다."


In [134]:
query = "교사와 학생 모델의 차이점은 무엇인가?"

response = hybrid_search(query, faq_retriever, chunk_retriever)
print(response)


📌 **FAQ 응답**
**Q:** 교사 모델의 편향이나 독성과 같은 결함이 학생 모델에 어떻게 전파되는지, 그리고 그 전파 수준을 수치화하여 관리할 수 있는 기술에는 어떤 것들이 있나요?

**A:** 답변: 
위험 전이(Risk Transfer) 현상은 대규모 언어 모델(LLM)을 기반으로 특정 목적의 소형 모델을 추출 및 학습시키는 과정에서 발생하는 핵심적인 윤리적, 기술적 과제로, 그 중요성이 날로 부각되고 있습니다. 교사 모델(Teacher Model)이 가진 편향성, 독성 발언 생성 경향, 사실관계 왜곡 등의 내재적 취약점이 학생 모델(Student Model)로 이전되는 이 현상은 단순히 모델의 크기를 줄이는 것이 안전성을 보장하지 않음을 시사합니다. 따라서, 이러한 위험이 어떤 메커니즘을 통해 전이되며, 이를 정량적으로 측정하고 통제할 수 있는 방법론을 확립하는 것은 책임감 있는 AI 개발을 위한 필수적인 연구 분야라 할 수 있습니다.

학생 모델이 교사 모델의 취약점을 상속받는 구체적인 메커니즘은 주로 지식 증류(Knowledge Distillation) 과정 자체에 기인합니다. 첫째, 가장 직접적인 경로는 ‘소프트 레이블(Soft Label)’ 모방입니다. 학생 모델은 교사 모델의 최종 출력(Hard Label)뿐만 아니라, 정답에 대한 확률 분포인 로짓(logits) 값까지 모방하도록 학습됩니다. 만약 교사 모델이 특정 편견에 기반하여 유해한 문장에 높은 확률을 할당한다면, 학생 모델은 이 확률 분포 자체를 학습 목표로 삼기 때문에 해당 편견을 그대로 재현하게 됩니다. 둘째, 교사 모델이 생성한 데이터를 학습에 사용하는 경우, 데이터 자체에 편향이 주입됩니다. 예를 들어, 특정 인구 집단에 대한 부정적인 내용을 담은 문장을 교사 모델이 생성하고 이를 학생 모델의 학습 데이터로 사용하면, 학생 모델은 이 편향된 데이터 분포를 통해 자연스럽게 취약점을 내재화합니다. 마지막으로, 잠재 공간(Latent Space)의 유사성 추구 역시 위험 전이의

In [ ]:
# 위의 하이브리드 단점 
정작 물어보는 질문에는 답변 x , faq에 나온 응답은 추가로 제공해야한다. 

교사와 학생 모델의 차이점에 대해 물어봤는데 비슷한 내용을 담은 faq만 반환했다. 
=> FAQ에서 관련성이 높은 "교사 모델 위험 전이" 질문을 매칭해버렸고, 그 답변만 반환

# 해결 1 
FAQ 답변 + Chunk 답변 동시에 제공
이렇게 하면 FAQ는 "참고 자료"로, Chunk는 "직접 답변"으로 보완 가능  

scoring을 엄격하게 해서 faq threshold를 0.9이상으로 높여, 똑같이 매칭되는 질문만 faq 답변 채택하고, FAQ는 참고용, Chunk가 항상 메인 답변

In [ ]:
import os
import google.generativeai as genai

# Gemini 초기화
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-2.5-pro")

FAQ_THRESHOLD = 0.9  # 엄격한 스코어 기준값

# -----------------------------
# 1. FAQ 기반 응답 (참고용)
# -----------------------------
def format_faq_answer(doc):
    q = doc.page_content
    a = doc.metadata.get("Answer", "")
    h1 = doc.metadata.get("Header 1", "")
    h2 = doc.metadata.get("Header 2", "")
    return f"""💡 **참고 FAQ**
**Q:** {q}

**A:** {a}

출처: {h1} > {h2}
"""

# -----------------------------
# 2. Chunk 기반 응답 (LLM 다듬기, 항상 메인)
# -----------------------------
def chunk_answer(query, retrieved_docs):
    docs_text = "\n\n".join(
        [
            f"[Chunk {doc.metadata.get('Chunk No')}] "
            f"(Header1: {doc.metadata.get('Header 1','')}, "
            f"Header2: {doc.metadata.get('Header 2','')}, "
            f"Header3: {doc.metadata.get('Header 3','')})\n"
            f"{doc.page_content}"
            for doc in retrieved_docs
        ]
    )

    prompt = f"""
    너는 연구 보고서를 요약하는 LLM 어시스턴트다. 
    아래 문서 조각들만 참고해서 질문에 대한 답변을 작성하라. 

    [요구사항]
    - 문서 내용만 활용할 것 (새로운 사실 생성 금지).
    - 답변은 3~4문단, 600~800자 내외로 정리.
    - 결론 문단은 반드시 '따라서, ~이다.' 또는 '결론적으로, ~라고 할 수 있다.' 형태로 마무리.
    - 답변 후 반드시 "출처" 섹션을 추가하여 사용한 Header1/2/3와 Chunk No를 나열할 것.

    질문: {query}

    문서 조각:
    {docs_text}
    """

    try:
        response = model.generate_content(
            prompt,
            generation_config={"max_output_tokens": 9000, "temperature": 0.5}
        )

        if response.candidates and response.candidates[0].content.parts:
            answer = "".join(
                [p.text for p in response.candidates[0].content.parts if hasattr(p, "text")]
            ).strip()
        else:
            answer = "[⚠️ 답변 없음: 모델이 응답을 생성하지 않음]"

    except Exception as e:
        answer = f"[⚠️ 오류 발생: {e}]"

    return "📑 **문서 기반 응답**\n" + answer

# -----------------------------
# 3. Hybrid Search
# -----------------------------
def hybrid_search(query, faq_retriever, chunk_retriever):
    # FAQ 검색
    faq_results = faq_retriever.get_relevant_documents(query)
    faq_output = None
    if faq_results:
        score = getattr(faq_results[0], "score", 0.0)
        if score >= FAQ_THRESHOLD:
            faq_output = format_faq_answer(faq_results[0])

    # Chunk 검색 (항상 메인)
    chunk_results = chunk_retriever.get_relevant_documents(query)
    chunk_output = chunk_answer(query, chunk_results) if chunk_results else "⚠️ Chunk 기반 결과 없음"

    # 합치기
    if faq_output:
        return f"{chunk_output}\n\n---\n\n{faq_output}"
    else:
        return chunk_output


SyntaxError: f-string expression part cannot include a backslash (185249893.py, line 84)

In [ ]:
# 일반 질문 테스트 
query = "교사와 학생 모델의 차이점은 무엇인가?"
response = hybrid_search(query, faq_retriever, chunk_retriever)
print(response) 


📑 **문서 기반 응답**
교사 모델과 학생 모델은 AI 모델 추출 기술에서 사용되는 개념으로, 크기, 목적, 학습 방식 및 잠재적 취약성에서 차이를 보입니다. 교사 모델은 일반적으로 매개변수가 많은 대규모 언어 모델로, 높은 비용과 긴 지연 시간이라는 실용적 한계를 가집니다. 반면, 학생 모델은 이러한 교사 모델의 유용한 기능과 행동을 모방하도록 훈련된 훨씬 작은 모델입니다. 모델 추출의 목적은 교사 모델의 운영 능력 상당 부분을 더 적은 계산 공간과 비용으로 구현하여 추론 속도와 운영 효율성을 높이는 데 있습니다. 학생 모델은 일반적인 성능을 지향하는 교사 모델과 달리, 특정 목적이나 제한된 지식 영역에 맞춰질 때 가장 효과적입니다.

두 모델의 핵심적인 차이는 학습 방식과 그에 따른 지식 및 위험의 전수 관계에서 비롯됩니다. 학생 모델은 처음부터 데이터를 학습하는 것이 아니라, 교사 모델이 학습한 데이터와 그 결과(예: 예측의 확률 분포)를 바탕으로 훈련됩니다. 이 과정에서 학생 모델은 교사 모델의 지식뿐만 아니라 내재된 편견, 결함, 보안 취약점까지 그대로 물려받게 됩니다. 예를 들어, 교사 모델이 개인 식별 정보를 유출할 가능성이 있다면, 그 학생 모델 역시 동일한 위험을 갖게 됩니다. 오히려 모델의 크기가 작고 함수가 단순해져 모델 반전과 같은 특정 보안 공격에 더 취약해질 수도 있습니다.

또한, 학생 모델은 스스로 맥락을 깊이 있게 이해하기보다 교사 모델의 사전 학습된 결론에 크게 의존하는 경향이 있습니다. 이는 모델 환각(Hallucination) 문제와 관련하여 새로운 위험을 야기할 수 있습니다. 일부 전문가는 교사 모델에 환각이 없다면 학생 모델도 안전할 것이라고 보지만, 다른 한편에서는 학생 모델의 작은 규모가 교사 모델의 모든 뉘앙스를 포착하지 못해 오류나 지나친 단순화를 유발하고, 이것이 새로운 형태의 환각으로 이어질 수 있다고 지적합니다. 이는 곧 정보 왜곡이나 AI 기반 악용 공격에 악용될 수 있는 위험으로 연결됩니다.

결론적으로

In [139]:
# 미리 생성한 예상 질문 테스트 
query = "LLM이 생성한 코드의 안정성을 보장하기 위해, 본문에서 언급된 '자동 계약 테스트', '점진적 린팅', '커밋 시 차이점 리뷰'를 CI/CD 파이프라인에 가장 효과적으로 통합할 수 있는 아키텍처는 무엇일까요? "
response = hybrid_search(query, faq_retriever, chunk_retriever)
print(response) 


📑 **문서 기반 응답**
LLM이 생성한 코드의 안정성을 보장하기 위해 본문에서 제시된 방안들을 CI/CD 파이프라인에 효과적으로 통합하는 아키텍처는 각기 다른 강점을 가진 LLM을 단계별로 활용하는 '멀티 모델 워크플로'입니다. 이 아키텍처는 LLM을 패턴 인식에는 뛰어나지만 책임감은 없는 '사전기억을 가진 인턴'처럼 다루어야 한다는 전제에서 출발합니다. LLM이 실패 경로를 건너뛰거나, 타입 검사 같은 안전장치를 임의로 비활성화하는 등의 문제를 일으킬 수 있기 때문에, 생성된 코드에 대한 강력한 검증 절차가 필수적입니다. 따라서 개발 프로세스의 각 단계에 검증 장치를 통합하는 구조가 필요합니다.

'바통터치' 방식으로도 불리는 이 워크플로는 특정 작업에 최적화된 모델을 순차적으로 사용하는 것이 핵심입니다. 예를 들어, UI 아이디어 탐색(GPT-4.1), 초기 사양서 작성(클로드), 기본 구조 스캐폴딩(제미나이 2.5), 핵심 로직 및 테스트 코드 작성(클로드 3.7), 그리고 최종 디버깅(o4-미니)으로 이어지는 단계별 접근 방식을 취합니다. 이러한 구조는 각 모델의 역할을 명확히 분리하여 성능을 극대화하고, 각 단계의 결과물을 다음 단계로 넘기기 전 검증을 수행할 수 있는 이상적인 통합 지점을 제공합니다.

이러한 멀티 모델 워크플로에 '자동 계약 테스트', '점진적 린팅', '커밋 시 차이점 리뷰'를 통합하는 것이 가장 효과적입니다. 예를 들어, 한 모델이 로직과 테스트 작성을 완료하면 해당 코드에 대해 자동 계약 테스트를 즉시 실행할 수 있습니다. 또한, 각 모델이 코드를 생성하거나 수정할 때마다 점진적 린팅을 적용하여 코드 품질을 일관되게 유지하고, 다음 단계로 넘어가기 전 커밋 시점에서 생성된 코드의 차이점을 면밀히 리뷰하여 잠재적 오류를 사전에 차단할 수 있습니다. 마지막 디버깅 단계에서는 이러한 테스트와 리뷰 과정에서 발견된 문제들을 집중적으로 수정하게 됩니다.

결론적으로, 각기 다른 LLM이 단계별로 역할을 수행하는 '바통터치' 방식의 

In [ ]:
# 예상 질문인데 chunk 기반으로 응답을 해서 아쉬움을 느낌, 예상 질문이 2문장이였는데 1문장만 테스트해본 결과 
# 참고 자료로 faq도 나왔으면 좋겠다 


In [ ]:
# 말씀하신 "참고 FAQ 여러 개" + "threshold 이중화"를 수행 

In [ ]:
import os
import google.generativeai as genai

# Gemini 초기화
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-2.5-pro")

FAQ_MAIN_THRESHOLD = 0.9   # 메인 질문 수준
FAQ_REF_THRESHOLD = 0.7    # 참고 질문 수준

# -----------------------------
# 1. FAQ 기반 응답 (참고용)
# -----------------------------
def format_faq_answer(doc):
    q = doc.page_content
    a = doc.metadata.get("Answer", "")
    h1 = doc.metadata.get("Header 1", "")
    h2 = doc.metadata.get("Header 2", "")
    return f"""💡 **참고 FAQ**
**Q:** {q}

**A:** {a}

출처: {h1} > {h2}
"""

# -----------------------------
# 2. Chunk 기반 응답 (LLM 다듬기, 항상 메인)
# -----------------------------
def chunk_answer(query, retrieved_docs):
    docs_text = "\n\n".join(
        [
            f"[Chunk {doc.metadata.get('Chunk No')}] "
            f"(Header1: {doc.metadata.get('Header 1','')}, "
            f"Header2: {doc.metadata.get('Header 2','')}, "
            f"Header3: {doc.metadata.get('Header 3','')})\n"
            f"{doc.page_content}"
            for doc in retrieved_docs
        ]
    )

    prompt = f"""
    너는 연구 보고서를 요약하는 LLM 어시스턴트다. 
    아래 문서 조각들만 참고해서 질문에 대한 답변을 작성하라. 

    [요구사항]
    - 문서 내용만 활용할 것 (새로운 사실 생성 금지).
    - 답변은 3~4문단, 600~800자 내외로 정리.
    - 결론 문단은 반드시 '따라서, ~이다.' 또는 '결론적으로, ~라고 할 수 있다.' 형태로 마무리.
    - 답변 후 반드시 "출처" 섹션을 추가하여 사용한 Header1/2/3와 Chunk No를 나열할 것.

    질문: {query}

    문서 조각:
    {docs_text}
    """

    try:
        response = model.generate_content(
            prompt,
            generation_config={"max_output_tokens": 9000, "temperature": 0.5}
        )

        if response.candidates and response.candidates[0].content.parts:
            answer = "".join(
                [p.text for p in response.candidates[0].content.parts if hasattr(p, "text")]
            ).strip()
        else:
            answer = "[⚠️ 답변 없음: 모델이 응답을 생성하지 않음]"

    except Exception as e:
        answer = f"[⚠️ 오류 발생: {e}]"

    return "📑 **문서 기반 응답**\n" + answer

# -----------------------------
# 3. Hybrid Search
# -----------------------------
def hybrid_search(query, faq_retriever, chunk_retriever):
    # FAQ 검색 (상위 3개까지 확인)
    faq_results = faq_retriever.get_relevant_documents(query)
    faq_outputs = []
    if faq_results:
        for doc in faq_results[:4]:
            score = getattr(doc, "score", 0.0)
            if score >= FAQ_REF_THRESHOLD:
                faq_outputs.append(format_faq_answer(doc))

    # Chunk 검색 (항상 메인)
    chunk_results = chunk_retriever.get_relevant_documents(query)
    chunk_output = chunk_answer(query, chunk_results) if chunk_results else "⚠️ Chunk 기반 결과 없음"

    # 합치기
    if faq_outputs:
        faq_block = "\n\n".join(faq_outputs)
        return f"{chunk_output}\n\n---\n\n{faq_block}"
    else:
        return chunk_output 


In [ ]:
# 미리 생성한 예상 질문 테스트 
query = "LLM이 생성한 코드의 안정성을 보장하기 위해, 본문에서 언급된 '자동 계약 테스트', '점진적 린팅', '커밋 시 차이점 리뷰'를 CI/CD 파이프라인에 가장 효과적으로 통합할 수 있는 아키텍처는 무엇일까요? "
response = hybrid_search(query, faq_retriever, chunk_retriever)
print(response)  


📑 **문서 기반 응답**
LLM이 생성한 코드의 안정성을 보장하기 위해서는 모델의 특성을 고려한 체계적인 아키텍처가 필요합니다. 문서에 따르면 LLM은 패턴 인식에는 뛰어나지만 책임감이 없어, 실패하는 경로를 건너뛰거나 의존성을 과도하게 설치하고, 타입 검사나 ESLint 같은 안전장치를 임의로 비활성화하는 경향이 있습니다. 이러한 특성 때문에 LLM을 '사전기억을 가진 인턴'처럼 다루어야 하며, 코드의 안정성을 확보하기 위해 '자동 계약 테스트', '점진적 린팅', '커밋 시 차이점 리뷰'는 필수적인 요소로 강조됩니다.

이러한 필수 검증 절차들을 CI/CD 파이프라인에 가장 효과적으로 통합할 수 있는 아키텍처는 여러 모델이 각자의 역할을 순차적으로 수행하는 '멀티 모델 워크플로', 즉 '바통터치' 방식입니다. 이 아키텍처는 UI 아이디어 탐색, 초기 사양서 작성, 스캐폴딩, 로직 구현, 디버깅 등 개발 단계를 세분화하고 각 단계에 최적화된 LLM을 할당하는 접근법입니다. 예를 들어, 한 모델이 컨트롤러 로직과 테스트 코드를 작성하는 단계를 완료하면, 그 결과물을 다음 모델이 이어받아 디버깅하고 테스트 통과를 책임지는 구조입니다.

이러한 단계별 전환 지점은 안정성 검증 절차를 통합하기에 최적의 환경을 제공합니다. 한 모델의 작업이 끝나고 다음 모델로 넘어가는 시점에 '커밋 시 차이점 리뷰'를 수행하여 변경 사항을 명확히 검토할 수 있습니다. 또한, 로직 구현이나 디버깅 단계가 완료될 때마다 파이프라인이 '자동 계약 테스트'를 실행하도록 구성할 수 있습니다. 각 모델이 코드를 생성하거나 수정할 때마다 '점진적 린팅'을 즉시 적용하여 코드 품질을 일관되게 유지하는 것 역시 이 구조 안에서 자연스럽게 이루어질 수 있습니다.

결론적으로, 각 모델이 자기 역할에 집중하고 다음 단계로 결과물을 넘기는 '바통터치' 방식의 아키텍처는 LLM 코드 생성의 각 단계마다 검증 게이트를 자연스럽게 마련해 줍니다. 이는 책임감 없는 '인턴' 같은 LLM의 결과물을 체계적으로 

In [ ]:
# threshold 이중화 후에도 동일한 결과 => FAQ 참고 블록이 여전히 붙지 않음 
# 리트리버에서 점수 제공하지 않아서 발생 가능성 높음 => 문서 점수 반환형식으로 변경 
faq_retriever.get_relevant_documents → 점수 없음
faq_retriever.vectorstore.similarity_search_with_score → (문서, 점수) 반환

In [ ]:
# 문제점 파악 : 사용한 임베딩 모델 불일치로 Threshold 값 적용 x 
faq는 CacheBackedEmbeddings 사용 
일반 chunk는 업스테이지임베딩 사용 

=> Upstage solar 임베딩은 한국어 최적화 → score 분포와 벡터 공간 자체가 다름
따라서 동일한 문장을 넣어도 FAQ retriever 쪽과 Chunk retriever 쪽이 다른 공간에서 검색 → 스코어 스케일도 달라짐
# FAQ와 검색 쿼리 모두 같은 임베딩(backbone)으로 처리해야 함 
왜 동일한 임베딩으로 통일해야해? 쿼리가 들어오면 벡터화시켜서 일반 chunk로 임베딩한 결과와 비교하고, 그 다음 faq로 임베딩한 결과와 비교하면 되잖아  

# => 한 쿼리를 같은 임베딩으로 두 벡터스토어에 동시에 던지면 문제가 생긴다는 게 포인트입니다.
지금 코드에서는 faq_retriever와 chunk_retriever 각각 잘 만들어 두셨죠.
하지만 질문 입력 쿼리가 내부적으로 어떤 임베딩 모델로 변환되는지는 retriever에 따라 달라요.
만약 둘 다 올바르게 분리해서 쓴다면 → FAQ는 OpenAI, Chunk는 Upstage 임베딩으로 각각 검색해야 정상적으로 매칭됩니다. 


아까 질문 다양화 한 작업이 헛고생이 되잖아 paraphrase 기억나지? 그러면 threshold 값 낮추고, 쿼리를 각각 임베딩 공간에서 검색해서 찾아주는 코드 작성해줘


In [ ]:
import os
import google.generativeai as genai

# Gemini 초기화
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-2.5-pro")

# Threshold 값 (현실적인 수준)
FAQ_THRESHOLD = 0.7   # FAQ 매칭 기준값 (낮춤)

# -----------------------------
# 1. FAQ 기반 응답 (참고용)
# -----------------------------
def format_faq_answer(doc):
    q = doc.page_content
    a = doc.metadata.get("Answer", "")
    h1 = doc.metadata.get("Header 1", "")
    h2 = doc.metadata.get("Header 2", "")
    return f"""💡 **FAQ 응답**
**Q:** {q}

**A:** {a}

출처: {h1} > {h2}
"""

# -----------------------------
# 2. Chunk 기반 응답 (LLM 다듬기, 항상 메인)
# -----------------------------
def chunk_answer(query, retrieved_docs):
    docs_text = "\n\n".join(
        [
            f"[Chunk {doc.metadata.get('Chunk No')}] "
            f"(Header1: {doc.metadata.get('Header 1','')}, "
            f"Header2: {doc.metadata.get('Header 2','')}, "
            f"Header3: {doc.metadata.get('Header 3','')})\n"
            f"{doc.page_content}"
            for doc in retrieved_docs
        ]
    )

    prompt = f"""
    너는 연구 보고서를 요약하는 LLM 어시스턴트다. 
    아래 문서 조각들만 참고해서 질문에 대한 답변을 작성하라. 

    [요구사항]
    - 문서 내용만 활용할 것 (새로운 사실 생성 금지).
    - 답변은 3~4문단, 600~800자 내외로 정리.
    - 결론 문단은 반드시 '따라서, ~이다.' 또는 '결론적으로, ~라고 할 수 있다.' 형태로 마무리.
    - 답변 후 반드시 "출처" 섹션을 추가하여 사용한 Header1/2/3와 Chunk No를 나열할 것.

    질문: {query}

    문서 조각:
    {docs_text}
    """

    try:
        response = model.generate_content(
            prompt,
            generation_config={"max_output_tokens": 9000, "temperature": 0.5}
        )

        if response.candidates and response.candidates[0].content.parts:
            answer = "".join(
                [p.text for p in response.candidates[0].content.parts if hasattr(p, "text")]
            ).strip()
        else:
            answer = "[⚠️ 답변 없음: 모델이 응답을 생성하지 않음]"

    except Exception as e:
        answer = f"[⚠️ 오류 발생: {e}]"

    return "📑 **문서 기반 응답**\n" + answer

# -----------------------------
# 3. Hybrid Search
# -----------------------------
def hybrid_search(query, faq_retriever, chunk_retriever):
    # FAQ 검색 (FAQ retriever는 OpenAI 기반 → paraphrase 포함 recall ↑)
    faq_results = []
    try:
        faq_results = faq_retriever.vectorstore.similarity_search_with_score(query, k=3)
    except Exception:
        faq_results = [(doc, 1.0) for doc in faq_retriever.get_relevant_documents(query)[:3]]

    faq_outputs = []
    for doc, score in faq_results:
        if score >= FAQ_THRESHOLD:
            faq_outputs.append(format_faq_answer(doc))

    # Chunk 검색 (항상 메인, Upstage 임베딩)
    chunk_results = chunk_retriever.get_relevant_documents(query)
    chunk_output = chunk_answer(query, chunk_results) if chunk_results else "⚠️ Chunk 기반 결과 없음"

    # 합치기
    if faq_outputs:
        faq_block = "\n\n".join(faq_outputs)
        return f"{chunk_output}\n\n---\n\n{faq_block}"
    else: 
        return chunk_output


In [150]:
# 미리 생성한 예상 질문 테스트 
query = "LLM이 생성한 코드의 안정성을 보장하기 위해, 본문에서 언급된 '자동 계약 테스트', '점진적 린팅', '커밋 시 차이점 리뷰'를 CI/CD 파이프라인에 가장 효과적으로 통합할 수 있는 아키텍처는 무엇일까요? "
response = hybrid_search(query, faq_retriever, chunk_retriever)
print(response) 


📑 **문서 기반 응답**
LLM이 생성한 코드의 안정성을 보장하기 위한 효과적인 아키텍처는 각기 다른 LLM이 특정 개발 단계를 순차적으로 담당하는 '멀티 모델 워크플로'를 기반으로 합니다. 이 접근법은 LLM을 패턴 인식에는 뛰어나지만 책임감은 없는 인턴사원처럼 다루어야 한다는 전제에서 출발합니다. LLM은 종종 실패 경로를 무시하거나, 불필요한 의존성을 추가하고, 타입 검사와 같은 안전장치를 임의로 비활성화하는 경향이 있기 때문에, 각 단계마다 강력한 검증 절차를 통합하는 것이 필수적입니다.

제시된 '바통터치' 방식의 워크플로는 이러한 검증 절차를 통합하기에 가장 이상적인 구조를 제공합니다. 예를 들어, 특정 LLM(예: 제미나이 2.5)이 코드의 전체적인 틀(스캐폴딩)을 생성하면, 다음 단계로 넘어가기 전에 '점진적 린팅'을 적용하여 코드 스타일과 잠재적 오류를 1차적으로 검사할 수 있습니다. 이후 다른 LLM(예: 클로드 3.7)이 세부 로직과 테스트 코드를 작성하면, CI 파이프라인 내에서 '자동 계약 테스트'를 실행하여 코드의 기능적 정확성과 안정성을 검증합니다. 이 테스트가 통과될 때까지 또 다른 디버깅 전문 LLM(예: o4-미니)이 코드를 수정하도록 하는 과정을 자동화할 수 있습니다.

이러한 단계별 검증을 거친 후, 최종적으로 코드를 커밋하기 전에는 '커밋 시 차이점 리뷰'를 통해 변경 사항을 최종 확인합니다. 이 아키텍처는 각 모델이 자신의 역할에만 집중하게 하여 성능을 극대화하고, 단계별 핸드오프 지점에 자동화된 품질 게이트(린팅, 테스트, 리뷰)를 배치함으로써 LLM이 생성한 코드의 신뢰도를 체계적으로 확보할 수 있습니다. 각 단계는 다음 단계의 입력값이 되므로, 계층적인 검증을 통해 최종 결과물의 안정성을 효과적으로 보장하게 됩니다.

결론적으로, 각기 다른 강점을 가진 LLM을 순차적으로 활용하는 '멀티 모델 워크플로'를 구축하고, 각 모델의 작업이 다음 단계로 넘어가는 전환 지점마다 자동 계약 테스트, 점진적 린팅, 커밋 시 차

In [ ]:
# # FAQ retriever가 질문 본문만이 아니라 Paraphrases까지 같이 임베딩하도록 한다. 
현재 faq 답변쌍이 나오지 않아서 코드 수정



In [152]:
import pandas as pd
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document

# 1. CSV 로드 (Paraphrases 포함)
df = pd.read_csv("techreader_data/content_based_FAQ_with_paraphrases.csv")

# 2. 임베딩 준비
embeddings = OpenAIEmbeddings()

# 3. FAQ 문서 생성 (질문 + Paraphrases 모두 page_content에 포함)
faq_docs = []
for _, row in df.iterrows():
    base_q = str(row["Question"]).strip()
    paras = []
    if "Paraphrases" in row and pd.notna(row["Paraphrases"]):
        try:
            paras = eval(row["Paraphrases"])  # 문자열 → 리스트 변환
        except:
            paras = [str(row["Paraphrases"])]
    # 원 질문 + 패러프레이즈 합치기
    combined_q = base_q + " " + " ".join(paras)
    faq_docs.append(Document(page_content=combined_q, metadata=row.to_dict()))

# 4. FAISS 인덱스 생성
faq_db = FAISS.from_documents(faq_docs, embeddings)
faq_retriever = faq_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

print("✅ FAQ retriever 준비 완료 (질문+paraphrase 기반)")


✅ FAQ retriever 준비 완료 (질문+paraphrase 기반)


In [153]:
import os
import google.generativeai as genai

# Gemini 초기화
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-2.5-pro")

# Threshold 값 (현실적인 수준)
FAQ_THRESHOLD = 0.7   # FAQ 매칭 기준값 (낮춤)

# -----------------------------
# 1. FAQ 기반 응답 (참고용)
# -----------------------------
def format_faq_answer(doc):
    q = doc.page_content
    a = doc.metadata.get("Answer", "")
    h1 = doc.metadata.get("Header 1", "")
    h2 = doc.metadata.get("Header 2", "")
    return f"""💡 **FAQ 응답**
**Q:** {q}

**A:** {a}

출처: {h1} > {h2}
"""

# -----------------------------
# 2. Chunk 기반 응답 (LLM 다듬기, 항상 메인)
# -----------------------------
def chunk_answer(query, retrieved_docs):
    docs_text = "\n\n".join(
        [
            f"[Chunk {doc.metadata.get('Chunk No')}] "
            f"(Header1: {doc.metadata.get('Header 1','')}, "
            f"Header2: {doc.metadata.get('Header 2','')}, "
            f"Header3: {doc.metadata.get('Header 3','')})\n"
            f"{doc.page_content}"
            for doc in retrieved_docs
        ]
    )

    prompt = f"""
    너는 연구 보고서를 요약하는 LLM 어시스턴트다. 
    아래 문서 조각들만 참고해서 질문에 대한 답변을 작성하라. 

    [요구사항]
    - 문서 내용만 활용할 것 (새로운 사실 생성 금지).
    - 답변은 3~4문단, 600~800자 내외로 정리.
    - 결론 문단은 반드시 '따라서, ~이다.' 또는 '결론적으로, ~라고 할 수 있다.' 형태로 마무리.
    - 답변 후 반드시 "출처" 섹션을 추가하여 사용한 Header1/2/3와 Chunk No를 나열할 것.

    질문: {query}

    문서 조각:
    {docs_text}
    """

    try:
        response = model.generate_content(
            prompt,
            generation_config={"max_output_tokens": 9000, "temperature": 0.5}
        )

        if response.candidates and response.candidates[0].content.parts:
            answer = "".join(
                [p.text for p in response.candidates[0].content.parts if hasattr(p, "text")]
            ).strip()
        else:
            answer = "[⚠️ 답변 없음: 모델이 응답을 생성하지 않음]"

    except Exception as e:
        answer = f"[⚠️ 오류 발생: {e}]"

    return "📑 **문서 기반 응답**\n" + answer

# -----------------------------
# 3. Hybrid Search
# -----------------------------
def hybrid_search(query, faq_retriever, chunk_retriever, show_scores=True):
    # -----------------
    # 1. FAQ 검색
    # -----------------
    faq_outputs = []
    faq_results = []
    try:
        faq_results = faq_retriever.vectorstore.similarity_search_with_score(query, k=5)
    except Exception:
        faq_results = [(doc, 1.0) for doc in faq_retriever.get_relevant_documents(query)[:5]]

    if show_scores:
        print("\n🔎 FAQ 검색 결과 점수 분포:")
        for doc, score in faq_results:
            print(f" - {score:.3f} | Q: {doc.page_content[:60]}...")

    for doc, score in faq_results:
        if score >= FAQ_THRESHOLD:
            faq_outputs.append(format_faq_answer(doc))

    # -----------------
    # 2. Chunk 검색
    # -----------------
    chunk_results = chunk_retriever.get_relevant_documents(query)
    chunk_output = chunk_answer(query, chunk_results) if chunk_results else "⚠️ Chunk 기반 결과 없음"

    # -----------------
    # 3. 최종 합치기
    # -----------------
    if faq_outputs:
        faq_block = "\n\n".join(faq_outputs)
        return f"{chunk_output}\n\n---\n\n{faq_block}"
    else:
        return chunk_output


In [ ]:
query = "LLM이 생성한 코드의 안정성을 보장하기 위해, 본문에서 언급된 '자동 계약 테스트', '점진적 린팅', '커밋 시 차이점 리뷰'를 CI/CD 파이프라인에 가장 효과적으로 통합할 수 있는 아키텍처는 무엇일까요?"

response = hybrid_search(query, faq_retriever, chunk_retriever, show_scores=True)
print("\n=== 최종 응답 ===\n")
print(response) 



🔎 FAQ 검색 결과 점수 분포:
 - 0.064 | Q: LLM이 생성한 코드의 안정성을 보장하기 위해, 본문에서 언급된 **'자동 계약 테스트', '점진적 린팅',...
 - 0.222 | Q: (AI 코딩 혼합 전략)** LLM 기반 코드 생성(Code Generation) 모델을 사내 개발 워크플로...
 - 0.228 | Q: LLM 체인/앙상블 아키텍처**: 단일 LLM의 한계를 넘어 복잡한 문제를 해결하기 위해, 여러 LLM을 순...
 - 0.230 | Q: LLM이 **'실패하는 경로를 스킵'**하는 경향을 보정하기 위해, 유닛 테스트 실패 결과(실패 로그, 스택...
 - 0.230 | Q: 내부 성능 평가 파이프라인 구축**: LLM 성능이 수 주 단위로 급변하는 상황에서, 외부 벤치마크에만 의존...

=== 최종 응답 ===

📑 **문서 기반 응답**
문서에 따르면, LLM이 생성한 코드의 안정성을 보장하기 위한 가장 효과적인 아키텍처는 전문화된 '멀티 모델 워크플로'를 통해 코드를 생성하고, 이를 CI/CD 파이프라인에 통합된 다단계 검증 프로세스로 검증하는 이중 구조입니다. LLM은 패턴 인식에는 뛰어나지만 책임감이 없어 실패 경로를 건너뛰거나, 불필요한 의존성을 추가하고, 타입 검사 같은 안전장치를 임의로 비활성화하는 경향이 있습니다. 이러한 문제를 해결하기 위해, LLM을 사전 지식은 있지만 책임감은 없는 인턴처럼 다루며 체계적인 검증 절차를 마련하는 것이 중요합니다.

이러한 아키텍처의 첫 단계는 각기 다른 LLM의 강점을 활용하는 '바통터치' 방식의 코드 생성 워크플로입니다. 예를 들어, UI 아이디어 탐색(GPT-4.1), 초기 사양서 작성(클로드), 기본 구조 스캐폴딩(제미나이), 핵심 로직 및 테스트 작성(클로드 3.7), 그리고 최종 디버깅(o4-미니) 등 각 단계에 최적화된 모델을 순차적으로 사용합니다. 이 방식은 각 모델이 특정 역할에 집중하게 하여 초기 코드의 품질을 높이고, 후속 검증 단계의 부담을 줄여줍니다

In [ ]:
# 해당 chunks기반으로 llm 지식 활용해서 예상질문 답변쌍을 만들었기 때문에 rag의 파이프라인인 구성 요소를 활용했다. (**************)

In [ ]:
# 리트리버의 사용 의문점이 들기 시작 
예상 질문 리스트에 없는 질문도, 리트리버가 있으면 임베딩 검색으로 대응 가능
즉, 예상 질문은 학습 보조, 리트리버는 실제 QA 엔진
질문이 심화형일수록 “실제 문서”를 근거로 해야 일관성과 정확도가 유지됨  => 제목 위주의 예상 질문 리스트만 제외 

예상 질문 + 답변(연구 보고서형): 학습·스터디·발표 자료 준비용
리트리버 기반 QA: 실제 서비스/프로덕션에서 사용자가 질문할 때 출처 포함 답변 제공 => 실사용 단계에 적합 (새로운 질문 대응 + 출처 추적)

In [ ]:
# 예상 질문 리스트에 대한 의문점 - 리트리버 기반 시스템을 더욱 견고하게 만든다. 
예상 질문 리스트를 미리 만드는 것은 리트리버 기반 QA 시스템을 구축하는 과정에서 매우 중요한 역할을 합니다. 
예상 질문 리스트는 시스템의 성능을 향상시키고, 초기 검증을 돕는 '보조 자료' 역할이다. 

1. 모델의 성능 평가 및 개선 가능 
예상 질문 리스트를 통해, 사용자가 질문할 시나리오를 미리 만들고 답변을 준비함으로써 모델이 다양한 질문에 얼마나 잘 응답하는지 테스트 가능하다. 
특정 질문에 대한 검색 정확도 recall이나 순위 rank를 측정하여 리트리버가 문서를 얼마나 잘 찾아내는지 평가할 수 있다. 

2. 리트리버 한계 보완
모든 질문에 완벽하게 대응하지 못할 수 있다. 이러한 예상 질문 리스트가 한계를 보완한다. 
매우 일반적이고 자주 묻는 질문 faq는 리트리버를 거치지 않고 미리 준ㅂ니한 답변을 즉시 제공하면 응답 속도 높이고 불필요한 리소스 사용 줄인다. 

3. 초기 시스템 검증 및 디버깅 
리트리버 기반 qa 를 처음 개발할 때, 예상 질문 리스트를 통해 초기 시스템의 동작을 검증하고 디버깅 가능하다. 
미리 정의된 질문 입력할 시, 시스템이 의도한 대로 동작하는지, 올바른 출처 문서를 검색하는지 확인

4. 사용자 경험 향상
예상 질문과 답변은 서비스를 처음 접하는 사용자에게 가이드 역할을 한다. 
자주 묻는 질문을 명시적으로 보여줌으로써 사용자가 시스템의 기능과 응답 범위를 이해하는데 도움을 준다. 

In [ ]:
# 지금 한 작업은 리트리버의 축소판으로, 리트리버가 하는 일을 예상 질문 생성 단계에서 미리 해놓은 것이다. 
리트리버의 필요성은 예상 질문 리스트에 없는 질문에 답변하기 위해서다. 
리트리버는 새로운 질문 → 관련 chunk 찾아서 답변 생성

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

query = "RAG의 최신 동향은 뭐야?"
docs = retriever.get_relevant_documents(query)

print("🔎 질문:", query)
for i, d in enumerate(docs, start=1):
    header1 = d.metadata.get("Header 1", "N/A")
    header2 = d.metadata.get("Header 2", "N/A")
    page = d.metadata.get("page", "알 수 없음")  # 라마파서에서 page_number 추가 가능
    print(f"\n=== 결과 {i} ===")
    print(d.page_content[:300], "...")
    print(f"출처: {header1} > {header2}, p.{page}")


# 9차 최종 정리할 것 
(md를 csv 변환한 것으로 사용할 것, 훨씬 더 깔끔한 출력)

# faq는 LLM 최신 동향 등의 개인적인 공부 용도로 사용하기 위해 만들었고,  
MD 파일 이용해서 벡터 스토어에 저장하고 FAQ는 학습 속도를 빠르게 하기 위해 미리 출력해주고 + 리트리버 성능 평가 


In [ ]:
LlamaParse + Gemini → PDF를 Markdown으로 변환 (규칙 적용)
LangChain Markdown Splitter → Markdown 파일을 헤더 단위로 분할

최종적으로 docs = LangChain Document 리스트, metadata에는 헤더 정보가 구조적으로 담김

In [ ]:
# PDF -> Markdown -> 헤더 분할 -> 임베딩 -> FAISS 검색 

In [151]:
import os
from llama_parse import LlamaParse
from langchain_core.documents import Document
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# ========= 1. PDF → Markdown 변환 ========= #
parser = LlamaParse(
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="gemini-2.5-pro",
    vendor_multimodal_api_key=os.environ["GOOGLE_API_KEY"],
    result_type="markdown",
    parsing_mode="Unstructured",
    language="ko",
    parsing_instruction="""
    당신은 PDF 문서를 구조화된 Markdown으로 변환하는 파서입니다.
    모든 텍스트를 가능한 보존하고, 주요 제목은 #, 소제목은 ## 로 변환하세요.
    """
)

file_path = r"techreader_data\LLM_TechLibrary.pdf"
parsed_docs = parser.load_data(file_path=file_path)

# LangChain Document 변환
docs = [doc.to_langchain_format() for doc in parsed_docs]

# Markdown 텍스트로 합치기
full_text = "\n\n".join([doc.page_content for doc in docs])
md_path = file_path.replace(".pdf", "_parsed.md")
with open(md_path, "w", encoding="utf-8") as f:
    f.write(full_text)

print(f"✅ Markdown 저장 완료: {md_path}")


# ========= 2. Markdown → 헤더 단위 분할 ========= #
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

with open(md_path, "r", encoding="utf-8") as f:
    markdown_text = f.read()

split_docs = markdown_splitter.split_text(markdown_text)

print(f"✅ 분할된 문서 수: {len(split_docs)}")
print("예시:", split_docs[0].page_content[:200], split_docs[0].metadata)


# ========= 3. FAISS 벡터 DB 구축 ========= #
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 가볍게 small 사용
db = FAISS.from_documents(split_docs, embeddings)

print("✅ FAISS 벡터DB 생성 완료")


# ========= 4. 검색 실행 ========= #
query = "LLM 기술 트렌드에 대해 요약해줘"
retrieved_docs = db.similarity_search(query, k=3)

print("\n📌 검색 결과:")
for i, doc in enumerate(retrieved_docs):
    print(f"\n---- 결과 {i+1} ----")
    print(doc.page_content[:300])
    print("메타데이터:", doc.metadata)


Started parsing the file under job_id e3a1141e-0878-4a6e-9187-aaf289c881c9
......✅ Markdown 저장 완료: techreader_data\LLM_TechLibrary_parsed.md
✅ 분할된 문서 수: 32
예시: IT WORLD CIO {}
✅ FAISS 벡터DB 생성 완료

📌 검색 결과:

---- 결과 1 ----
LLM 코딩은 여전히 사람의 검토가 필요하다. 네 모델 모두 때때로 다음과 같은 행동을 한다 :  
Deep Dive 19
메타데이터: {'Header 1': 'AI 코딩, LLM 혼합 전략이 답이다', 'Header 3': '배포 전 꼭 유념해야 할 마지막 회의적 시각'}

---- 결과 2 ----
**Tech Trend**
- "아는 것만 아는" LLM, 오히려 혁신을 저해한다
- LLM을 학습한 추출 모델, 작아도 위험은 동일  
**Tech Guide**
- LLM 한계 극복을 위한 RAG의 역할과 최신 동향
- 잊어버려야 할 것은 잊는 LLM이 필요한 시점
- AI 코딩, LLM 혼합 전략이 답이다  
무단 전재 재배포 금지  
본 PDF 문서는 IDG Korea의 자산으로, 저작권법의 보호를 받습니다. IDG Korea의 허락 없이 PDF 문서를 온라인 사이트 등에 무단 게재, 전재하거나 유포할 수 없습니다.  
Te
메타데이터: {'Header 1': '“LLM 이후를 설계하다”', 'Header 2': '생성형 AI의 과제와 대안 찾기'}

---- 결과 3 ----
LLM은 훈련에 막대한 자원과 시간이 소요된다. 엔비디아 H200과 같은 최첨단 서버 GPU 수백 대를 사용해 몇 달 동안 훈련해야 하는 경우도 있다. 이런 이유로 LLM을 완전히 최신 상태로 유지하기 위해 처음부터 다시 훈련하는 것은 사실상 불가능하다. 대신 더 적은 비용이 드는 방법으로 기존 모델을 최신 데이터로 미세 조정(fine-tuning)하는 방식이 활용된다.  
그러나 미세 조정에